In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
# from compass_smartsheet import CompassSmartsheet
# from ftplib import FTP
import cisco_fiscal_calendar as fc
import hashlib

In [2]:
pd.__version__

'1.2.2'

In [3]:
today_timestamp = dt.datetime.now().isoformat()[:19]

In [4]:
def convert_nan_to_int_to_str(dataframe_cell):
        if pd.isnull(dataframe_cell):
            return pd.NA
        else:
            try:
                return str(int(dataframe_cell))
            except:
                return pd.NA

In [5]:
# def convert_requestid_to_str(requestid_cell):
#     return str(int(requestid_cell))

### Prepare the Partners Dictionary

In [6]:
partner = pd.read_excel(r'C:\Users\phsheari\Documents\Compass Partners\Compass Partners Hash.xlsx')

In [7]:
compass_partner_names = partner['CompassPartnerName']
standardname = partner['StandardName']

In [8]:
partner_dict = {}
for k,v in zip(compass_partner_names,standardname):
    partner_dict[k] = v

In [9]:
# partner_dict

In [10]:
sss_map = pd.read_excel(r'C:\Users\phsheari\Documents\Francisco\SSS_GEO Mapping.xlsx')

In [11]:
sss = list(sss_map['SSS'])

In [12]:
geo = list(sss_map['SL3'])

In [13]:
sssgeo_dict = {}
for k, v in zip(sss,geo):
    sssgeo_dict[k] = v

In [14]:
# sssgeo_dict

#### Bring in FINBI Accrued Results to verify booked records are tagged as '5 - Closed Won'

In [15]:
finbi_data_types = {'Sales Level 1':str,                     
                    'Sales Level 2':str, 
                    'Sales Level 3':str, 
                    'CAV ID':str, 
                    'CAV Name':str,
                    'GUID':str, 
                    'GUID Name':str, 
                    'Partner':str,
                    'Booked Deal ID':str, 
                    'CX Product Category':str, 
                    'Fiscal Year':str,
                    'Fiscal Quarter ID':str, 
                    'Fiscal Week ID':str, 
                    'Fiscal Week of Year':str, 
                    'ReportWeek':str, 
                    'ProductType':str,
                    'TCV':np.float32, 
                    'ACV':np.float32, 
                    'Request ID':str, 
                    'Readout_Date':np.datetime64, 
                    'Campaign Type':str,
                    'Campaign':str, 
                    'Target Campaign Flag':str, 
                    'Partner Deal':str,
                    'Service Source':str,
                    'Compass GUID Name':str,
                    'Compass Deal ID':str,
                    'Compass Advisor':str,
                    'Readout Delivery Status':str,
                    'Items Recommended':str,}

In [16]:
finbi_path = r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.xlsx'
finbi_sheetname = 'YTD Transactions'
finbi_columns = ['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name',
       'GUID', 'GUID Name', 'Partner','Booked Deal ID', 'CX Product Category', 'Fiscal Year',
       'Fiscal Quarter ID', 'Fiscal Week ID', 'Fiscal Week of Year', 'ReportWeek', 'ProductType',
       'TCV', 'ACV', 'Request ID', 'Readout_Date', 'Campaign Type',
       'Campaign', 'Target Campaign Flag', 'Partner Deal','Service Source','Compass GUID Name','Compass Deal ID','Compass Advisor','Readout Delivery Status','Items Recommended', ]
finbi_df = pd.read_excel(finbi_path, sheet_name = finbi_sheetname, usecols=finbi_columns, dtype=finbi_data_types)
finbi_df['Request ID'] = finbi_df['Request ID'].apply(convert_nan_to_int_to_str)
finbi_df['Readout_Date'] = finbi_df['Readout_Date'].apply(lambda x: pd.to_datetime(x).date() if x != pd.NA else x)

finbi_df.to_pickle(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.pkl')
finbi_df.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results_Copy.xlsx', index=False)

#maven_fields_to_drop = ['Ind','Compass Exists?','Compass Claim?','Notes','Respondant_File','AddIncre']
maven_version = finbi_df #.drop(columns=maven_fields_to_drop)
maven_version.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\Maven\Compass Bookings.xlsx', index=False)

finbi_requests = list(finbi_df.loc[finbi_df['Request ID'].notnull()]['Request ID'].unique())

In [17]:
# finbi_df.columns
finbi_requests.sort()

In [18]:
# finbi_requests

### Start the pull from SMARTSHEET

In [19]:
start = dt.datetime.now()

# # The Compass Smartsheet object init's a class which retrieves, cleans, & modifies records to prepare for presentation. 
# # It also unmarks, then remarks the Duplicate DealID Request records

# dflist = []
# for i in range(1,4,1):                  # This paginates the CompassSmartsheet Object, pulling 10k records per page
#     cs = CompassSmartsheet(page = i)
#     tmpdf = cs.get_report_dataframe()   # this takes the dataframe object from cs and stores it in tmpdf
#     dflist.append(tmpdf)                # appends each tmpdf to dflist, for future concatenation
    
# nonbooked = pd.concat(dflist)

# finish = dt.datetime.now()
# elapsed = (finish - start)
# print(f'Compass object was created in {elapsed} minutes. It shows {nonbooked.shape[0]} rows in the dataframe.')

In [20]:
nonbooked = pd.read_pickle(r'C:\Users\phsheari\Documents\Compass SNOWFLAKE\data_download\result.pkl')

In [21]:
today_file = today_timestamp.replace('-','').replace('T','_').replace(':','')

In [22]:
today_file

'20220303_181048'

In [23]:
nonbooked.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37376 entries, 0 to 37375
Columns: 107 entries, Request ID to WORKING STATUS
dtypes: bool(9), datetime64[ns](7), float64(13), int64(16), object(62)
memory usage: 28.3+ MB


### Push Booked deals to a reference file for corrective measures

In [24]:
nickname_files_path = r'C:\Users\phsheari\Documents\Compass Nickname'
nonbooked['Nickname'] = pd.NA
USECOLS = ['Request ID']
DTYPES = {'Request ID':str}

entries = os.listdir(nickname_files_path)
for f_name in entries:
    tempdf = pd.read_excel(nickname_files_path + '\\' + f_name, usecols = USECOLS, dtype=DTYPES)
    id_list = list(tempdf['Request ID'].unique())
    nonbooked.loc[nonbooked['Request ID'].isin(id_list),'Nickname'] = f_name[:-5]
    print(f'Nickname is {f_name[:-5]} and its count is {len(id_list)}')

Nickname is Mirthas Top68 and its count is 52
Nickname is US Commercial Missed Attach Mari 20211130 and its count is 376


In [25]:
nonbooked.loc[nonbooked['Nickname'].notnull()][['Request ID','Create Date','Customer Name','Campaign','Nickname','WORKING STATUS']]

,Request ID,Create Date,Customer Name,Campaign,Nickname,WORKING STATUS
214,107585,2021-11-24 20:55:02.766,CURTISS-WRIGHT CORPORATION,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
215,107584,2021-11-24 20:54:57.403,PENN NATIONAL GAMING,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
216,107578,2021-11-24 20:54:28.685,PRISMA HEALTH,REACTIVE,US Commercial Missed Attach Mari 20211130,DECLINED
217,107579,2021-11-24 20:54:33.427,DOMTAR PAPER COMPANY,REACTIVE,US Commercial Missed Attach Mari 20211130,DECLINED
218,107580,2021-11-24 20:54:38.550,XPO LOGISTICS,REACTIVE,US Commercial Missed Attach Mari 20211130,DECLINED
...,...,...,...,...,...,...
37371,107566,2021-11-24 20:53:28.591,SYNCRO CORPORATION,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
37372,107567,2021-11-24 20:53:33.745,ENCOMPASS HEALTH CORPORATION,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
37373,107568,2021-11-24 20:53:38.865,SLIDELL MEMORIAL HOSPITAL,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE
37374,107569,2021-11-24 20:53:44.214,MILLIPORE SIGMA,REACTIVE,US Commercial Missed Attach Mari 20211130,READOUT DONE


In [26]:
# booked_requestids = nonbooked.loc[nonbooked['TARGET_REQUEST_ID'].isin(finbi_requests),['Request ID','Forecast Stage', 'Customer Name','Deal ID','Compass Campaign Name']]

In [27]:
# booked_requestids.sort_values(by=['Sheet Name','Request ID'], inplace=True)

In [28]:
# booked_requestids.shape[0]

In [29]:
# booked_requestids.loc[booked_requestids['Forecast Stage']!='5 - Closed Won'].to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\REQUEST IDS TO BOOK.xlsx', index=False)
# booked_requestids.loc[booked_requestids['Forecast Stage']!='5 - Closed Won'].to_pickle(r'C:\Users\phsheari\Documents\FIN BI Data\REQUEST IDS TO BOOK.pkl')

### Begin to Inspect, Clean, & Curate the remaining dataframe

In [30]:
nonbooked['DELIVERABILITY'].value_counts()

Good                 30655
Insufficient Data     6721
Name: DELIVERABILITY, dtype: int64

In [31]:
hyper_csv_file_path_tracker = r'C:\Users\phsheari\Documents\Compass_hyper_csvs\compass_tracker_hyper.csv'
excel_file_path_tracker = r'C:\Users\phsheari\Documents\Compass Trackers\ConsolidatedReport_' + today_file + '.xlsx'
excel_file_path_sfdc_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'
excel_file_path_tableau = r'C:\Users\phsheari\Documents\Compass Tableau Data\ConsolidatedReport_' + today_file + '.xlsx'

In [32]:
# nonbooked = nonbooked.reindex(columns = booked.columns)
# nonbooked = nonbooked.loc[~(nonbooked['Forecast Stage']=='5 - Closed Won')]

In [33]:
#Determine what is undeliverable so you can avoid or remove any pipeline values in those records

undeliverable_requests = list(nonbooked.loc[(nonbooked['INSUFFICIENT_DATA_COUNT']==1)]['Request ID'])

nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Forecast Stage']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Forecast Status']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Opportunity Name']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Projected Booking ($,000)']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Target Fiscal Month']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Target Fiscal Quarter']=pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(undeliverable_requests),'Target Fiscal Year']=pd.NA

In [34]:
def update_did_dupechk(did, dupechk):
    if pd.isnull(did):
        return False
    elif did == None:
        return False
    elif did in [-44444,99999,9999,22222]:
        return False
    else:
        return dupechk

In [35]:
# Initialize the nonbooked dataframe Dupechk field to 0. Sort the Deal ID Subset dataframe to determine which rows contain duplicate deal IDs
nonbooked['Dupechk'] = 0

did_subset = nonbooked.loc[(nonbooked['INSUFFICIENT_DATA_COUNT']==0),['Request ID','Create Date','Customer Name', 'Deal ID','Forecast Stage']].copy()
did_subset.sort_values(['Deal ID','Request ID','Create Date'], ascending=[True, False, False], ignore_index=False, inplace=True)
did_subset['Dupechk'] = did_subset.duplicated(['Deal ID'], keep='first')  # Chronologically sets the first deal id occurance to False, all after to True
did_subset['Dupechk'] = did_subset.apply(lambda x: update_did_dupechk(x['Deal ID'], x['Dupechk']), axis=1)

In [36]:
dupes_requestid_list = list(did_subset.loc[(did_subset['Dupechk']==True) & ~(did_subset['Forecast Stage']=='5 - Closed Won')]['Request ID'])

In [37]:
len(dupes_requestid_list)

3751

In [38]:
# did_subset.to_excel(r'C:\Users\phsheari\Desktop\Dupes.xlsx', index=False)

In [39]:
# nonbooked.info()

In [40]:
# booked.info()

In [41]:
# nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list)]   ### Sanity Check

In [42]:
# Set the duplicate deal ID values so that pipeline values are not duplicated
nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Dupechk'] = 1
nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Forecast Stage'] = '7 - Duplicate Request'
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Forecast Status'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Opportunity Name'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Projected Booking ($,000)'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Target Fiscal Month'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Target Fiscal Quarter'] = pd.NA
#nonbooked.loc[nonbooked['Request ID'].isin(dupes_requestid_list),'Target Fiscal Year'] = pd.NA

In [43]:
#df = pd.concat([booked, nonbooked]) #### [booked, nonbooked]
df = nonbooked

In [44]:
df.shape

(37376, 110)

### Change the field name "BDM Assigned" to "Compass Advisor"

In [45]:
df.rename(columns={'BDM Assigned': 'Compass Advisor'}, inplace=True)

### Read in the Compass Advisor Email Lookup Table to resolve proper names

In [46]:
advisor_emails = pd.read_excel(r'C:\Users\phsheari\Documents\Compass Advisors\Advisor Email Translation.xlsx', usecols=['cecid','name', 'source'])

In [47]:
advisor_email = dict(zip(advisor_emails.cecid, advisor_emails.name))

In [48]:
advisor_vendor = dict(zip(advisor_emails.cecid, advisor_emails.source))

In [49]:
def get_advisor_name_from_cecid(advisor):
    if pd.isnull(advisor) or advisor == None or advisor.lower()=='nan':
        return None
    if advisor=='Expired' or advisor=='N/A':
        return 'Expired'
    if advisor.lower().endswith('cisco.com'):
        pieces = advisor.lower().split('@')
        cecid = pieces[0]
        actual_name = advisor_email.get(cecid,'Term')
        return actual_name 
    else:
        return advisor

In [50]:
def get_advisor_vendor_from_cecid(advisor):
    if pd.isnull(advisor) or advisor == None or advisor.lower()=='nan':
        return None
    if advisor=='Expired' or advisor=='N/A':
        return None
    if advisor.lower().endswith('cisco.com'):
        pieces = advisor.lower().split('@')
        cecid = pieces[0]
        vendor_name = advisor_vendor.get(cecid, None)
        if vendor_name:
            return vendor_name.upper()
        else:
            return None
    else:
        return None

In [51]:
df['NewAdvisor'] = df['Compass Advisor'].apply(lambda x: get_advisor_name_from_cecid(x))

In [52]:
df['Vendor'] = df['Compass Advisor'].apply(lambda x: get_advisor_vendor_from_cecid(x))

In [53]:
df['Concierge Advisor Name'] = df['CONCIERGE_ADVISOR_ASSIGNED'].apply(lambda x: get_advisor_name_from_cecid(x))

In [54]:
df['Concierge Advisor Name'].unique()

array([None, 'Alicia Biersteker', 'Adrian Machado', 'Martin Coup',
       'Atif Ahmad', 'Mark Vodka', 'Emma Baird', 'Tracy Turner',
       'Peter Wittenstrom', 'Stacey Edwards', 'Houman Asefi',
       'Sangeetha Sampath', 'Clare Fagan', 'Brenda Buck', 'Jenee St John',
       'Bill Black-Hogins', 'Jenee St. John', 'Alicia Bierstkeker',
       'Stacey Edwards,', 'DUPLICATE of SAME record',
       'Jenee St John <jestjohn@cisco.com>', 'x', 'Term',
       'Alicia Bierstker', 'Alicia'], dtype=object)

In [55]:
df['Compass Advisor'] = df['NewAdvisor']

In [56]:
# df['Compass Advisor'].unique()

In [57]:
df.drop(columns=['NewAdvisor'], inplace=True)

In [58]:
df['Vendor'].unique()

array(['MODIS INC', 'NUB78', 'ADECCO UK LTD', None, 'SERVICESOURCE',
       'ADECCO AISAPAC', 'ADECCO AUSTRALIA', 'CISCO'], dtype=object)

### Fix/Clean Customer Names

In [59]:
name_correction = {'201820184589':'NIPPON TELEGRAPH AND TELEPHONE EAST CORPORATION'}

In [60]:
def correct_customer_name(df):
    reqid = df['TARGET_REQUEST_ID']
    if reqid in name_correction.keys():
        return name_correction.get(reqid)
    else:
        return df['Customer Name'].strip()

In [61]:
# df['Customer Name'] = df['Customer Name'].astype(str)
# df['Customer Name'] = df.apply(lambda x: correct_customer_name(x), axis=1)

In [62]:
df['Customer Name'] = df['Customer Name'].str.lower().str.replace('\^_ inc',' inc').str.replace('\^_ ',', ').str.replace(', ',' ').str.replace(' inc.',' inc ').str.replace('\* ','').str.strip().str.upper()

<ipython-input-62-c82f039ad48b>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Customer Name'] = df['Customer Name'].str.lower().str.replace('\^_ inc',' inc').str.replace('\^_ ',', ').str.replace(', ',' ').str.replace(' inc.',' inc ').str.replace('\* ','').str.strip().str.upper()


In [63]:
def squeeze_customer_name(customer_name):
    if customer_name.startswith('THE '):
        customer_name = customer_name[4:]
    customer_name = customer_name.replace(',',"").replace('[ACTIVE]','').replace('-','').replace('_','').replace('.','').replace(' ','').strip()
    return customer_name

In [64]:
df['Squeezed'] = df['Customer Name'].apply(lambda x: squeeze_customer_name(x))

In [65]:
# df['Squeezed']

In [66]:
df['Prepend'] = df['Squeezed'].apply(lambda x: x[:2].upper())

In [67]:
# df.loc[(df['Customer Name'].str.contains('choctaw', case=False))] #df['GUID'].isnull()) | 

### Standardize Campaign Name to Uppercase

In [68]:
df['Campaign'] = df['Campaign'].str.upper()

In [69]:
df.loc[df['Campaign'].str.contains('missed attach', na=False, case=False),'Campaign'] = df.loc[df['Campaign'].str.contains('missed attach', na=False, case=False), 'Campaign'].apply(lambda x: x.replace('MISSED ATTACH','COVERAGE PROGRAM'))

In [70]:
df.loc[df['Campaign'].str.contains('coverage p', na=False, case=False)][['Request ID','Customer Name','Campaign']]

,Request ID,Customer Name,Campaign
25,111388,WA-AUBURN SCHOOL DISTRICT 408,COVERAGE PROGRAM - TELEMETRY
26,111352,UT-SALT LAKE VALLEY EMERGENCY COMMUNICATIONS C...,COVERAGE PROGRAM - TELEMETRY
27,111346,UT-CITY OF RIVERTON,COVERAGE PROGRAM - TELEMETRY
28,111386,VOYAGER LEARNING,COVERAGE PROGRAM - TELEMETRY
29,111342,KIRBY CORPORATION,COVERAGE PROGRAM - TELEMETRY
...,...,...,...
37151,103950,ENDURANCE REINSURANCE CORPORATION OF AMERICA,COVERAGE PROGRAM - TELEMETRY
37155,104005,INSURITY LLC,COVERAGE PROGRAM - TELEMETRY
37156,104021,TIMEX CORPORATION,COVERAGE PROGRAM - TELEMETRY
37157,104041,LUME STRATEGIES INC,COVERAGE PROGRAM - TELEMETRY


In [71]:
# df.loc[df['Requester Name'].str.lower().isin(confirm_exclusions)][['Request ID','Requester Name']]

### Clean up issues with Requester Name

In [72]:
confirm_exclusions = ['kenpatton@cisco.com','kpatton@cisco.com','lsweidan@cisco.com','escudder@cisco.com', 'rosecam@cisco.com']

In [73]:
def swap_requester_name_with_notified(requester_name, distribution_list, campaign_name):
    if requester_name:
        requester_name = str(requester_name.lower())
    if distribution_list:
        distribution_list = str(distribution_list)
    if (campaign_name != 'nan' or pd.notnull(campaign_name)):
        campaign_name = str(campaign_name).lower()
     
    if ('target' in campaign_name or 'tac lead' in campaign_name or 'success tracks' in campaign_name or 'major' in campaign_name) and (requester_name.lower() in confirm_exclusions and pd.notnull(distribution_list)):
        requester_name = distribution_list
        return requester_name.upper()
    else:
        return requester_name.upper()
    

In [74]:
df.loc[df['Requester Name'].notnull(),'Requester Name'] = df.loc[df['Requester Name'].notnull()].apply(lambda x: swap_requester_name_with_notified(x['Requester Name'],x['DISTRIBUTION_LIST'], x['Campaign']), axis=1)

### Sanity Check

In [75]:
# df.loc[(df['Compass Campaign Name']=='BCS 3.0 TARGETING') & (df['Who Should be Notified On Completion of Analysis'].notnull())][['Request ID','Customer Name','Requester Name','Who Should be Notified On Completion of Analysis']]

In [76]:
# name_counts = pd.DataFrame(df['Requester Name'].value_counts())

In [77]:
# name_counts.reset_index(inplace=True)

In [78]:
# name_counts

In [79]:
# df.loc[df['Requester Name']=='nan']

### Set Recipient Confirmation Status & Clean Requester Name

In [80]:
def set_recipient_confirmation_field(requester_name, compass_url, oa_url, distribution_list):
    #requester_name = str(requester_name)

    if pd.isnull(requester_name) or requester_name == 'N/A' or requester_name == 'nan' or type(requester_name) == bool or requester_name.lower().startswith('kpatton'):
        return "NOT CONFIRMED"
    
    elif (pd.isnull(compass_url) and pd.isnull(oa_url)):
          return 'NOT CONFIRMED'
    
    elif requester_name.lower() in confirm_exclusions and not pd.isnull(distribution_list):
        return "CONFIRMED"
    
    elif requester_name.lower() in confirm_exclusions  and pd.isnull(distribution_list):
        return "NOT CONFIRMED"
    
    else:
        return "CONFIRMED"

In [81]:
df['Recipient Confirmation'] = df.apply(lambda x: set_recipient_confirmation_field(x['Requester Name'], x['COMPASS URL'], x['OA URL'], x['DISTRIBUTION_LIST']), axis=1)

In [82]:
df['Recipient Confirmation'].value_counts()

CONFIRMED        26689
NOT CONFIRMED    10687
Name: Recipient Confirmation, dtype: int64

In [83]:
working_status_list = ['ASSIGNED','BOOKED','EXCEPTION_REVIEW','READOUT DONE','READOUT POSTPONED','SCHEDULED','UNASSIGNED']

In [84]:
# df.loc[(df['Request FY']=='2022') & (df['Report FY']=='2022') & (df['WORKING STATUS'].isin(working_status_list)) & (df['Recipient Confirmation']=='NOT CONFIRMED')][['Requester Name','DISTRIBUTION_LIST','Recipient Confirmation']]

### Note: The Requester Name is acting as if there is hidden / blank / characters. Test deeply!!!

In [85]:
# requester_list = list(df['Requester Name'])

In [86]:
# len(requester_list)

In [87]:
# confirm_list = list([set_recipient_confirmation_field(x) for x in requester_list])

In [88]:
# len(confirm_list)

In [89]:
# kk = pd.DataFrame(confirm_list)

In [90]:
# kk.rename(columns={0:'Recipient Confirmation'}, inplace=True)

In [91]:
# kk.value_counts()

In [92]:
df.loc[df['Recipient Confirmation'].isin(['NOT CONFIRMED'])][['Request ID','Create Date','Customer Name','GUID','Deal ID','Requester Name','Campaign']].shape

(10687, 7)

In [93]:
# def fix_requester_name(requester_name, sss_confirmation):
#     requester_name = str(requester_name)
#     sss_confirmation = str(sss_confirmation)
    
#     if sss_confirmation.lower() == "not confirmed":
#         return requester_name
    
#     if pd.isnull(requester_name) or requester_name == 'nan':
#         return pd.NA
    
#     if requester_name.lower().endswith("@cisco.com"):
#             return requester_name
#     else:    # requester name does not end in "@cisco.com"
#         if not ',' in requester_name:   # would not be construed as a list of items
#                 return requester_name + "@cisco.com"
#         else:
#             requester_name = requester_name.replace(';',',')
#             alist = list(requester_name.split(','))
#             checklist = []
#             for n in alist:
#                 checklist.append(n + "@cisco.com")
#             return ', '.join(checklist)

In [94]:
# df['Requester Name'] = df.apply(lambda x: fix_requester_name(x['Requester Name'],x['Recipient Confirmation']), axis = 1)

In [95]:
# df['Requester Name'].value_counts()

In [96]:
# df['Requester Name'] = df['Requester Name'].apply(lambda x: None if x == 'nan@cisco.com' else x)

### Assign Compass Advisor name

In [97]:
def clean_bdm_assigned(bdm_assigned):
    if pd.isnull(bdm_assigned):
        return None
    elif bdm_assigned.lower() == 'nan':
        return 'Expired'
    else:
        return bdm_assigned.title()

In [98]:
df['Compass Advisor'] = df['Compass Advisor'].apply(clean_bdm_assigned)

In [99]:
# df.loc[df['Recipient Confirmation'] == 'Confirmed',['Request ID','Requester Name','Compass Campaign Name','Recipient Confirmation']].shape

In [100]:
# df['Compass Advisor'].unique()

In [101]:
confirmed = df.loc[(df['Recipient Confirmation']=='Confirmed') & ~(df['Requester Name'].str.lower().isin(confirm_exclusions)) & (df['Requester Name'].notnull())][['Request ID', 'Create Date','Customer Name','Requester Name','Campaign','Sales Level 2','Sales Level 3','Recipient Confirmation', 'Compass Advisor','Readout_Date']]

In [102]:
# Sanity Check -- Test if we show all the "Confirmed" recipients for readouts on Targeting Campaign leads
confirmed

,Request ID,Create Date,Customer Name,Requester Name,Campaign,Sales Level 2,Sales Level 3,Recipient Confirmation,Compass Advisor,Readout_Date


In [103]:
def populate_sl3_from_requestername(requester_name):
    if requester_name in sssgeo_dict.keys():
        try:
            return sssgeo_dict.get(requester_name)
        except:
            return 'No SL3 Available'

confirmed['Sales Level 3'] = confirmed.apply(lambda x: populate_sl3_from_requestername(x['Requester Name']), axis=1)

In [104]:
confirmed = confirmed[['Request ID', 'Create Date', 'Customer Name', 'Requester Name','Recipient Confirmation','Campaign', 'Sales Level 2', 'Sales Level 3',]]

### Fix the Sales Level3 value for certain Requester Names

In [105]:
# df.loc[df['Request ID'].isin(list(confirmed['Request ID'])),'Lvl3'] = df.loc[df['Request ID'].isin(list(confirmed['Request ID']))].apply(lambda x: populate_sl3_from_requestername(x['Requester Name']), axis=1)

### Standardize the Partner Names from the partner_dict hash

In [106]:
PartnerNames = list(df.loc[df['Partner Name'].notnull()]['Partner Name'].unique())

In [107]:
PartnerNames.sort()

In [108]:
# PartnerNames

In [109]:
# partner_dict
def update_partner_names(partner_name):
    if partner_name == np.nan or partner_name is None or pd.isnull(partner_name):
        return None
    else:
        PN = partner_name.lower().replace('. ','').replace('.','')
        newname = partner_dict.get(PN, PN.upper())
        return newname

In [110]:
df['Partner Name'] = df['Partner Name'].apply(update_partner_names)

### Apply Fiscal Period Dating to the Create Date and Readout Date fields

In [111]:
### Fix Injection & Readout Date for 10 Missed Attach records
coverage_at_renewal_q1_list = ['104315','104316','104317','104318','104319','104320','104321','104322','104323','104324']    #['104315','104316','104317','104318','104319','104320','104321','104322','104323','104324']    #[104315,104316,104317,104318,104319,104320,104321,104322,104323,104324]   

In [112]:
df.loc[df['Request ID'].isin(coverage_at_renewal_q1_list), 'Create Date'] = pd.to_datetime('20211101T1903', format='%Y-%m-%d %H:%M:%S', errors='ignore')

In [113]:
df.loc[df['Request ID'].isin(coverage_at_renewal_q1_list), 'Readout_Date'] = pd.to_datetime('20211103', format='%Y-%m-%d', errors='ignore')

In [114]:
df.loc[df['Request ID'].isin(coverage_at_renewal_q1_list) ][['Request ID','Create Date','Readout_Date']]

,Request ID,Create Date,Readout_Date
33588,104315,2021-11-01 19:03:00,2021-11-03
33589,104316,2021-11-01 19:03:00,2021-11-03
33590,104317,2021-11-01 19:03:00,2021-11-03
33591,104324,2021-11-01 19:03:00,2021-11-03
33596,104318,2021-11-01 19:03:00,2021-11-03
33597,104319,2021-11-01 19:03:00,2021-11-03
33598,104320,2021-11-01 19:03:00,2021-11-03
33599,104323,2021-11-01 19:03:00,2021-11-03
33600,104321,2021-11-01 19:03:00,2021-11-03
33601,104322,2021-11-01 19:03:00,2021-11-03


In [115]:
df.loc[df['Campaign'].str.contains('coverage at', na=False, case=False) & (df['Readout_Date'] < '2021-10-31')]

,Request ID,Create Date,Created By,Request Type,Modified,Modified By,Date Completed,TARGET_REQUEST_ID,LAST_UPDATED,QA Approve,...,UNCOVERED_OPPTY_VALUE,WORKING STATUS,Nickname,Forecast Stage,Dupechk,Vendor,Concierge Advisor Name,Squeezed,Prepend,Recipient Confirmation


In [116]:
def get_fiscal_half(fiscal_quarter):
    if pd.isnull(fiscal_quarter):
        return pd.NA
    elif fiscal_quarter == '1' or fiscal_quarter == '2':
        return '1'
    else:
        return '2'

In [117]:
df['Request FY'] = df['Create Date'].apply(fc.get_fiscal_year)

In [118]:
df['Request FQ'] = df['Create Date'].apply(fc.get_fiscal_quarter)

In [119]:
df['Request FM'] = df['Create Date'].apply(fc.get_fiscal_month)

In [120]:
df['Request FWOFM'] = df['Create Date'].apply(fc.get_fiscal_week_of_fiscal_month)

In [121]:
df['Request FQ'].unique()

array(['3', '2', '4', '1'], dtype=object)

In [122]:
df['Request FH'] = df['Request FQ'].apply(get_fiscal_half)

In [123]:
df['Request FWOFQ'] = df['Create Date'].apply(fc.get_fiscal_week_of_fiscal_quarter)

In [124]:
df['Request FWOFY'] = df['Create Date'].apply(fc.get_fiscal_week_of_fiscal_year)

In [125]:
df['Readout_Date'] = pd.to_datetime(df['Readout_Date'])

In [126]:
df['Readout FY'] = df['Readout_Date'].apply(fc.get_fiscal_year)
df['Readout FQ'] = df['Readout_Date'].apply(fc.get_fiscal_quarter)
df['Readout FM'] = df['Readout_Date'].apply(fc.get_fiscal_month)

In [127]:
df['Readout FWOFM'] = df['Readout_Date'].apply(fc.get_fiscal_week_of_fiscal_month)

In [128]:
df['Readout FWOFQ'] = df['Readout_Date'].apply(fc.get_fiscal_week_of_fiscal_quarter)

In [129]:
df['Readout FWOFY'] = df['Readout_Date'].apply(fc.get_fiscal_week_of_fiscal_year)

In [130]:
df['Readout FH'] = df['Readout FQ'].apply(get_fiscal_half)

In [131]:
df['Request FH'].unique()

array(['2', '1'], dtype=object)

### Set the Campaign Type field from the campaign names

In [132]:
def set_campaign_type(campaign_name):
    if pd.isnull(campaign_name) or campaign_name == '':
        return 'NO CAMPAIGN'
    
    elif campaign_name.lower() == 'test' or campaign_name.lower() == 'accelerator' or campaign_name.lower() == 'proactive - ignore' \
    or campaign_name.upper() == 'IAP – STEPHAN' or campaign_name.upper() == 'IAP - STEPHAN' or 'research' in campaign_name.lower():
        return 'NONREPORTING/TESTS'
    
    elif 'cxib scale' in campaign_name.lower():
        return 'COVERAGE AT RENEWAL'
    
    elif 'cxib' in campaign_name.lower():
        return 'CXIB'    
    
    elif 'coverage at renewal' in campaign_name.lower():
        return 'COVERAGE AT RENEWAL'
    
    elif campaign_name.upper() == 'COVERAGE PROGRAM - TELEMETRY':
        return 'TELEMETRY'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('sntc tac leads' in campaign_name.lower() or 'tac sntc' in campaign_name.lower() or campaign_name.upper()=='PROACTIVE - TAC LEAD'):
        return 'TAC'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('proactive attach' in campaign_name.lower() or 'proactive sntc' in campaign_name.lower()):
        return 'PROACTIVE COVERAGE'
    
    elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()) and ('field' in campaign_name.lower() or 'reactive' in campaign_name.lower()):
        return 'FIELD'
    
    #elif ('missed attach' in campaign_name.lower() or 'coverage program' in campaign_name.lower()):
    #    return 'PROACTIVE COVERAGE'
    
    elif 'proactive - refresh bdm' in campaign_name.lower():   #'PROACTIVE - AMERICAS - IRR EXPAND',
        return 'NONREPORTING/TESTS'
        
    elif 'proactive' in campaign_name.lower() or campaign_name.upper() in ('PROACTIVE - REFRESH','PROACTIVE - CS RESEARCH'):   #'PROACTIVE - AMERICAS - IRR EXPAND',
        return 'PROACTIVE_OLD'
    
    elif 'proactive tac' in campaign_name.lower():
        return 'TAC'
    
    elif 'reactive' in campaign_name.lower() or campaign_name.upper() in ('UKI - SUZI','UKI -- SUZI','PROACTIVE - PARTNER.1','PROACTIVE - PARTNER.2') or campaign_name.upper() == 'PROACTIVE - REFRESH BDM':   #'PROACTIVE - AMERICAS - IRR EXPAND',
        return 'FIELD'
    
    elif campaign_name.upper() == 'TELEMETRY':
        return 'TARGETING'
    
    elif 'bcs' in campaign_name.lower() or 'success' in campaign_name.lower() or 'solution' in campaign_name.lower() or 'sspt' in campaign_name.lower() or 'st target' in campaign_name.lower() or 'ctu' in campaign_name.lower() or 'target' in campaign_name.lower():
        return 'TARGETING'    

In [133]:
df['Campaign Type'] = df['Campaign'].apply(lambda x: set_campaign_type(x))

In [134]:
# df['Campaign Type'].unique()
df.loc[df['Campaign'].str.contains('CXIB', na=False, case=False)]['Campaign Type'].unique()

array(['CXIB'], dtype=object)

In [135]:
df.loc[df['Campaign Type'].isnull()]['Campaign'].unique()

array(['TEST REQUEST'], dtype=object)

### Set the Items Recommended fields

In [136]:
def bcs_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'BCS' in items_recommended:
        return 'BCS'
    else:
        return None

In [137]:
# df['Items Recommended'].unique()

In [138]:
df['IR_BCS'] = df['Items Recommended'].apply(bcs_recommended)

In [139]:
def ec_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'EC-' in items_recommended:
        return 'EC'
    else:
        return None

In [140]:
df['IR_EC'] = df['Items Recommended'].apply(ec_recommended)

In [141]:
def dnac_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'DNA' in items_recommended:
        return 'DNAC'
    else:
        return None

In [142]:
df['IR_DNAC'] = df['Items Recommended'].apply(dnac_recommended)

In [143]:
def sspt_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'SSPT' in items_recommended:
        return 'SSPT'
    else:
        return None

In [144]:
df['IR_SSPT'] = df['Items Recommended'].apply(sspt_recommended)

In [145]:
def st_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'ST' in items_recommended:
        return 'ST'
    else:
        return None

In [146]:
df['IR_ST'] = df['Items Recommended'].apply(st_recommended)

In [147]:
def sntc_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'SNTC' in items_recommended:
        return 'SNTC'
    else:
        return None

In [148]:
df['IR_SNTC'] = df['Items Recommended'].apply(sntc_recommended)

In [149]:
def irr_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'IRR' in items_recommended:
        return 'IRR'
    else:
        return None

In [150]:
df['IR_IRR'] = df['Items Recommended'].apply(irr_recommended)

In [151]:
def productrefresh_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'PR' in items_recommended:
        return 'PR'
    else:
        return None

In [152]:
df['IR_PR'] = df['Items Recommended'].apply(productrefresh_recommended)

In [153]:
def brw_recommended(items_recommended):
    if pd.isnull(items_recommended):
        return None
    elif 'BRW' in items_recommended:
        return 'BRW'
    else:
        return None

In [154]:
df['IR_BRW'] = df['Items Recommended'].apply(brw_recommended)

In [155]:
# def target_campaigns_priority_recommendation(items_recommended):
#     if pd.isnull(items_recommended):
#         return None
#     elif 'BCS' in items_recommended:
#         return 'BCS'
#     elif 'EC' in items_recommended:
#         return 'BCS'
#     elif 'IRR' in items_recommended:
#         return 'BCS'
#     elif 'ST' in items_recommended:
#         return 'ST'
#     elif 'SSPT' in items_recommended:
#         return 'SSPT'
#     elif 'SNTC' in items_recommended:
#         return 'SNTC'

In [156]:
# df['TC_Priority_Recommend'] = df['Items Recommended'].apply(target_campaigns_priority_recommendation)

In [157]:
# def create_index_id(requestID, sheetname):
#     if pd.isnull(requestID):
#         return None
#     else:
#         indexid = requestID + '_' + sheetname
#         return indexid

In [158]:
# df['IndexID'] = df.apply(lambda x: create_index_id(x['Request ID'], x['Sheet Name']), axis = 1)

In [159]:
# df['IndexID']

In [160]:
requesters_to_remove_2021 = list(['alcastri@cisco.cm@cisco.com','alcastri@cisco.com','anamoren@cisco.com','antmills@cisco.com','avillalo@cisco.com','boenglis@cisco.com',
                                 'borirodr@cisco.com','braolive@cisco.com','brbuck@cisco.com','camruiz@cisco.com','fpizano@cisco.com', 'jbaldera@cisco.com','jcollie2@cisco.com',
                                 'josguti2@cisco.com','jszmak@cisco.com','kpatton@cisco.com','kywatts@cisco.com','lmaldon@cisco.com', 'lsweidan@cisco.com' , 'nilosada@cisco.com',
                                 'nnino@cisco.com','qhopkins@cisco.com','stotero@cisco.com', 'wzeliger@cisco.com','caltree@cisco.com','obustosm@cisco.com','antchris@cisco.com',
                                 'dadiaspe@cisco.com','jforgy@cisco.com','jguggehe@cisco.com','stehartm@cisco.com','abdulahm@cisco.com','pamohan@cisco.com','charpost@cisco.com',
                                 'mragam@cisco.com','javalenc@cisco.com','escudder@cisco.com'])

In [161]:
campaigns_to_remove_from_count_2021 = list(['ACCELERATOR','IAP - STEPHAN','PROACTIVE - AMERICAS - F20Q2-3 RENEW (140)','PROACTIVE - AMERICAS - IRR EXPAND', 'PROACTIVE - AMERICAS - US COMM.1','PROACTIVE - AMERICAS - US COM.1',
                                       'PROACTIVE - BCS 3.0', 'PROACTIVE - CAP.1','PROACTIVE - CS RESEARCH', 'PROACTIVE - EMEAR.BETA','PROACTIVE - IGNORE','PROACTIVE - PARTNER.1','PROACTIVE - PARTNER.2', 'PROACTIVE - PRESALES ALIGN PITSTOP EFFORT',
                                       'PROACTIVE - REFRESH LIST', 'PROACTIVE - SW CONFORMANCE', 'PROACTIVE - SW CONFORMANCE.2', 'PROACTIVE - TAC LEAD', 'PROACTIVE - USPS.1','REACTIVE - APJC GC ENT', 'REACTIVE - APJC.1','REACTIVE - APJC.BETA',
                                       'REACTIVE - ASEAN.1', 'REACTIVE - CANSAC', 'REACTIVE - CISCOREADY','REACTIVE - EMEAR-PILOT','REACTIVE - GROWTH INITIATIVE','REACTIVE - KYLE WATTS','REACTIVE - RENEWALS Q3 KSO','PROACTIVE REFRESH BDM',   ### THE 412 HEALTHCARE REQUESTS
                                       'TEST', 'TEST REQUEST'])

In [162]:
requesters_to_remove_2022 = list(['lsweidan@cisco.com','nenglema@cisco.com','escudder@cisco.com', 'jestjohn@cisco.com' ])

In [163]:
temp_campaigns_to_remove_from_count = list([ #'BCS 3.0 TARGETING HEALTHCARE CAMPAIGN',
                                           'PROACTIVE - RESEARCH',
                                           'REACTIVE - CXIB',
                                           'REACTIVE - GROWTH INITIATIVE',
                                           'TEST REQUEST'])

In [164]:
def remove_from_the_items_to_be_counted(requester_name, compass_campaign_name, SL3, created_date, date_completed):   # Anything FALSE is intended to be removed. #When this gets into Tableau, we only want to select the TRUE values
    if pd.isnull(requester_name) or pd.isnull(date_completed):
        return False
    elif created_date < pd.to_datetime('20210801', format='%Y%m%d') and (requester_name in requesters_to_remove_2021 or compass_campaign_name in campaigns_to_remove_from_count_2021):
        return False
       
    elif created_date >= pd.to_datetime('20210801', format='%Y%m%d') and (requester_name in requesters_to_remove_2022 or compass_campaign_name in temp_campaigns_to_remove_from_count):
        return False
    
    else:
        return True

In [165]:
# pd.to_datetime('20210801', format='%Y%m%d')

In [166]:
df['REMOVE_FROM_COUNT'] = df.apply(lambda x: remove_from_the_items_to_be_counted(x['Requester Name'],x['Campaign'], x['Sales Level 3'], x['Create Date'], x['Date Completed']), axis=1)

In [167]:
df.loc[(df['Campaign'].str.contains('IDENTIFIED RISKS', na=False, case=False)) & (df['Sales Level 1']=='AMERICAS') & (df['REMOVE_FROM_COUNT']==True) ].groupby(['Campaign','Sales Level 3','REMOVE_FROM_COUNT']).count()['Request ID']

Campaign                                                 Sales Level 3      REMOVE_FROM_COUNT
BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RISKS (NEW)  ASP TELCO MOBILE   True                  1
                                                         ASP US SEGMENTS    True                 29
                                                         ASP_CANADA         True                  2
                                                         ASP_LATAM          True                  1
                                                         COMMERCIAL CAN     True                  7
                                                         ENTERPRISE CA      True                  8
                                                         GES WEST           True                 69
                                                         PUBLIC SECTOR CAN  True                 15
                                                         SLED-EAST AREA     True                 26
      

In [168]:
df.loc[df['REMOVE_FROM_COUNT'] == True].groupby(['Requester Name','Campaign Type','REMOVE_FROM_COUNT']).count()['Request ID']

Requester Name      Campaign Type  REMOVE_FROM_COUNT
AABUL@CISCO.COM     FIELD          True                  1
AADAY@CISCO.COM     TARGETING      True                  2
AAFRICA@CISCO.COM   FIELD          True                  2
AAKKAWI@CISCO.COM   TAC            True                  1
AALAJAJI@CISCO.COM  TARGETING      True                  6
                                                        ..
YUKOHAYA@CISCO.COM  TARGETING      True                  1
YVWOIT@CISCO.COM    TARGETING      True                  1
ZALGHAMD@CISCO.COM  FIELD          True                  1
ZISTEPHE@CISCO.COM  FIELD          True                 32
ZJANICEL@CISCO.COM  FIELD          True                  1
Name: Request ID, Length: 1800, dtype: int64

In [169]:
df.shape

(37376, 140)

### RR (internal) field, a.k.a. Readout Required:
#### If a record is OLDER than 6 months, a readout will NOT be done.
#### If a record's Compass Campaign Name is in an exclusion list (TBD), a readout will not be done.
#### If either of the above conditions are met, set a "NO" value in the "RR (internal)" field, else set a "YES" value there

In [170]:
campaign_exclusion_list = ['PROACTIVE - REFRESH LIST','REACTIVE - GROWTH INITIATIVE','PROACTIVE - CS RESEARCH','PROACTIVE - IGNORE','PROACTIVE - TAC LEAD','TEST','TEST REQUEST','REACTIVE - BDM','ACCELERATOR','PROACTIVE - PRESALES ALIGN PITSTOP EFFORT','REACTIVE - CISCOREADY','REACTIVE - PARTNER','REACTIVE - PARTNER.1','REACTIVE - PARTNER.2',]

In [171]:
campaign_inclusion_list = ['AMERICAS SP','REACTIVE - UKI – SUZI','REACTIVE','REACTIVE - AOJC','REACTIVE - AMERICAS','BCS TARGETING AMERICAS','BCS TARGETING - UKI','REACTIVE - EMEAR','REACTIVE -GESW','BCS TARGETING AMERICAS-CANADA','BCS 3.0 TARGETING',]

In [172]:
requester_exclusion_list = ['wzeliger@cisco.com','mragam@cisco.com','caltree@cisco.com','obustosm@cisco.com','antchris@cisco.com',
                            'dadiaspe@cisco.com','lsweidan@cisco.com','jforgy@cisco.com','jguggenhe@cisco.com', 'stehartm@cisco.com',
                            'abdulham@cisco.com','pamohan@cisco.com','charpost@cisco.com','javalenc@cisco.com','kkaler@cisco.com']

In [173]:
lookback_date = dt.datetime.now().date() + dt.timedelta(days = -92)

In [174]:
lookback_date

datetime.date(2021, 12, 1)

In [175]:
### Sanity Indicator - Are there records without the Request ID?
df.loc[df['Request ID'].isnull()][['Created By','Customer Name','GUID','Deal ID','Requester Name','Campaign',]] #.groupby(['Created By'])

,Created By,Customer Name,GUID,Deal ID,Requester Name,Campaign


In [176]:
df = df.loc[df['Request ID'].notnull()]

In [177]:
not_in_list = ['test','test request','accelerator','proactive - ignore']

In [178]:
df = df.loc[~df['Campaign'].str.lower().isin(not_in_list)]

In [179]:
# guidtest1 = '6829690,35452880,89286617'
# guidtest2 = '512349548'

In [180]:
# print(guidtest1.split(','), guidtest2.split(','))

In [181]:
### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName

def make_fake_guid(guid, cavid, customername):
    import pdb
    if pd.isnull(guid):
        if pd.isnull(cavid) or cavid == -999:
            return str(customername[:10])
        else:
            if not isinstance(cavid, int):
                cavid = cavid.split(',')
                return str(cavid[0])
    elif not isinstance(guid,int):
        try:
            if isinstance(guid,float):
                guid = str(int(guid))
                return guid
            
            elif isinstance(guid,str):
                guid = guid.split(',')
                guid = str(guid[0])
                return guid
        except:
            return str(guid)
    else:
        return str(guid)

In [182]:
# make_fake_guid(np.nan, guidtest1, 'Jac')

In [183]:
# ### Start with GUID. If GUID is None, use CAV ID, if CAV ID, is None, use CustomerName, if GUID is bad or a list with unacceptable alpha lexicon values, use CustomerName
# ### OLD METHOD from Smartsheet

# def make_fake_guid(guid, cavid, customername):
#     if pd.isnull(guid):
#         if pd.isnull(cavid):
#             return str(customername[:10])
#         elif isinstance(cavid, float):
#             return str(int(cavid))
#         elif ',' in cavid:
#             quicklist = list(cavid.split(','))
#             return str(int(quicklist[0]))
#         else:
#             return str(int(cavid))
#     elif type(guid) == float:
#         return str(int(guid))
    
#     elif isinstance(guid, str):
#         quicklist = list(cavid.split(','))
#         return str(int(quicklist[0]))
        
#         if int(guid):
#             return str(int(guid))
#         else:
#             return str(customername[:10])
    
#     elif ',' in guid:
#         quicklist = list(guid.split(','))
#         return str(int(quicklist[0]))
    
#     elif ';' in guid:
#         quicklist = list(guid.split(';'))
#         return str(int(quicklist[0]))
    
#     elif '\r\n' in guid:
#         quicklist = list(guid.split('\r\n'))
#         return str(int(quicklist[0]))
    
#     elif guid == '-':
#         return str(customername[:10])
        
#     elif str(guid).isalpha():
#         return str(customername[:10])
    
#     elif ' ' in guid:
#         quicklist = list(guid.split(' '))
#         if quicklist[0].isalpha():
#             return str(customername[:10])
#         else:
#             return str(int(quicklist[0]))
    
#     elif len(str(guid)) > 12:
#         return str(guid)[:12]
    
#     else:
#         return str(guid)

In [184]:
df['FakeID'] = df.apply(lambda x: make_fake_guid(x['GUID'], x['CAV ID'], x['Squeezed']), axis=1)

In [185]:
df[['Request ID','GUID','FakeID']]

,Request ID,GUID,FakeID
0,112652,"3967128,73909",3967128
1,111949,179365505,179365505
2,111948,"254033997,122674843,144688312,187444876,236144...",254033997
3,111647,40456091,40456091
4,111648,41318866,41318866
...,...,...,...
37371,107566,10368781,10368781
37372,107567,320903967,320903967
37373,107568,3773236,3773236
37374,107569,2134315,2134315


In [186]:
def make_hash_entity(prehash_entity, bdm, readoutdate, items_recommded):    #compassurl, 
    entity_to_hash = str(prehash_entity).upper() + '_' + str(bdm).upper() + '_' + str(readoutdate) + '_' + str(items_recommded).upper()   #str(compassurl).upper() + '_' + 
    return entity_to_hash

In [187]:
df['Prehash_Entity'] = df.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [188]:
df.loc[df['Prehash_Entity'].notnull()][['Request ID','Create Date','Customer Name','GUID','FakeID','CAV ID','Prepend','Prehash_Entity']]

,Request ID,Create Date,Customer Name,GUID,FakeID,CAV ID,Prepend,Prehash_Entity
0,112652,2022-02-17 22:26:29.919,ENBRIDGE INC,"3967128,73909",3967128,NaN,EN,EN3967128
1,111949,2022-02-07 04:16:24.656,TRAFIKVERKET,179365505,179365505,NaN,TR,TR179365505
2,111948,2022-02-07 03:57:07.084,CENTRAL GOVERNMENT UK,"254033997,122674843,144688312,187444876,236144...",254033997,NaN,CE,CE254033997
3,111647,2022-01-28 07:47:12.834,FRIEDE SPRINGER GMBH & CO. KG,40456091,40456091,562367,FR,FR40456091
4,111648,2022-01-28 07:47:15.071,GLOBAL MOBILITY HOLDING BV,41318866,41318866,173452,GL,GL41318866
...,...,...,...,...,...,...,...,...
37371,107566,2021-11-24 20:53:28.591,SYNCRO CORPORATION,10368781,10368781,NaN,SY,SY10368781
37372,107567,2021-11-24 20:53:33.745,ENCOMPASS HEALTH CORPORATION,320903967,320903967,NaN,EN,EN320903967
37373,107568,2021-11-24 20:53:38.865,SLIDELL MEMORIAL HOSPITAL,3773236,3773236,NaN,SL,SL3773236
37374,107569,2021-11-24 20:53:44.214,MILLIPORE SIGMA,2134315,2134315,NaN,MI,MI2134315


## Set the Deal ID on the Refresh Record based upon the Previous Request ID

In [189]:
# request_dealid_dict = {}
req_dealids = df[['Request ID','Deal ID']].copy()
req_dealids = req_dealids.dropna(how='any', axis=0)
req_dealids.set_index('Request ID', inplace=True)
request_dealid_dict = req_dealids.to_dict()

In [190]:
# request_dealid_dict['Deal ID']['2021202114287']

In [191]:
def get_the_deal_id(requestid):
    dealid = None
    if requestid in request_dealid_dict['Deal ID']:
        dealid = request_dealid_dict['Deal ID'][requestid]
        if isinstance(dealid, list):
            return dealid
        elif isinstance(dealid, str) and not dealid.isalpha():
            return dealid
        elif isinstance(dealid, float):
            return int(dealid)
        else:
            return dealid
    else:
        return None

In [192]:
# get_the_deal_id('202120211223236')
# df['Prior Deal ID'] = df['Previous Request ID'].apply(get_the_deal_id)

In [193]:
# df.loc[df['Prior Deal ID'].notnull() & df['Compass Campaign Name'].str.contains('close the gap', case=False)][['Request ID','Previous Request ID','Prior Deal ID','Deal ID','Compass Campaign Name']]

In [194]:
df['Entity_to_hash'] = pd.NA

In [195]:
def is_this_a_targeting_campaign(compass_campaign_name):
    if pd.isnull(compass_campaign_name):
        return False
    else:
        compass_campaign_name = str(compass_campaign_name)
    
    if 'major' in compass_campaign_name.lower():
        return True
    elif 'sspt' in compass_campaign_name.lower():
        return True
    elif 'solution' in compass_campaign_name.lower():
        return True
    elif 'success' in compass_campaign_name.lower():
        return True
    elif 'st target' in compass_campaign_name.lower():
        return True
    elif 'ctu' in compass_campaign_name.lower():
        return True
    elif 'bcs' in compass_campaign_name.lower():
        return True
    elif 'partner' in compass_campaign_name.lower():
        return False
    else:
        return False

In [196]:
df['is_Targeting_Campaign'] = df['Campaign'].apply(is_this_a_targeting_campaign)

In [197]:
def set_targeting_campaign_type(compass_campaign_name):
    compass_campaign_name = str(compass_campaign_name)
    
    if 'major' in compass_campaign_name.lower():
        return "MAJORS"
    elif 'sspt' in compass_campaign_name.lower():
        return "SSPT"
    elif 'solution' in compass_campaign_name.lower():
        return "SSPT"
    elif 'success' in compass_campaign_name.lower():
        return "SUCCESSTRACKS"
    elif 'st target' in compass_campaign_name.lower():
        return "SUCCESSTRACKS"
    elif 'ctu' in compass_campaign_name.lower():
        return "SNTC"
    elif 'bcs' in compass_campaign_name.lower():
        return "BCS"
    elif 'partner' in compass_campaign_name.lower():
        return 'PARTNER'
    else:
        return "NON-TARGETING"

In [198]:
df['Targeting Campaign Type'] = df['Campaign'].apply(set_targeting_campaign_type)

In [199]:
# def set_record_type(previous_request_id):
#     if not pd.isnull(previous_request_id):
#         return 'Refreshed'
#     else:
#         return 'Original'

In [200]:
# df['Record Type'] = df['Previous Request ID'].apply(set_record_type)

### We are Hashing the Company ID, the Compass Advisor, the Readout_Date/Scheduled, and the Items Recommended

In [201]:
df.loc[(df['Items Recommended'].notnull()) | ((df['Readout Delivery Status'].notnull()) & (~df['Readout Delivery Status'].isin(['Declined','Postponed']))) & (df['Compass Advisor'].notnull()),'Entity_to_hash'] = \
df.loc[(df['Items Recommended'].notnull()) | ((df['Readout Delivery Status'].notnull()) & (~df['Readout Delivery Status'].isin(['Declined','Postponed']))) & (df['Compass Advisor'].notnull())].apply(lambda x: make_hash_entity(x['Prehash_Entity'], x['Compass Advisor'], x['Readout_Date'], x['Items Recommended']), axis=1)

In [202]:
def hash_the_readout(entity_to_hash):
    #if pd.isnull(entity_to_hash):  #or entity_to_hash == 'NA'
    if not entity_to_hash:
        return pd.NA
    else:
        entity_to_hash = entity_to_hash.encode()
        hashed = hashlib.md5(entity_to_hash)
        hashed = str(hashed.hexdigest())
        return hashed

In [203]:
df['HashedReadout'] = pd.NA

In [204]:
df.loc[df['Entity_to_hash'].notnull(),'HashedReadout'] = df.loc[df['Entity_to_hash'].notnull()].apply(lambda x: hash_the_readout(x['Entity_to_hash']), axis =1)

In [205]:
df.loc[df['Entity_to_hash'].notnull()][['Request ID','HashedReadout']]

,Request ID,HashedReadout
1,111949,f0aaa98058ee49a8ec07b2a85c0a5736
33,111333,75ef3002862e107f66e9add00fb55a43
39,111236,2091d0524ee0bd5bffe0755fcf15dfc7
40,111181,c460162b4a36ecf4b12e70daa98c23ed
41,111126,ff73c3631e4b63fa902f39b78375e521
...,...,...
37371,107566,8b7b0aebae31b2a3130b9a1251b603cd
37372,107567,dcbfcc3664ab6063160cf23817ae8636
37373,107568,e51a7718bad6cd5bf5a2caad877af536
37374,107569,23880ed49b4c61890731eee5eac8dfed


In [206]:
### Checking for Incomplete Readout Records

df.loc[df['Entity_to_hash'].notnull() & (df['Items Recommended'].notnull() & (df['Readout Delivery Status'].isnull() | df['Readout_Date'].isnull()))]\
[['Request ID','Create Date','Customer Name','GUID','Partner Name','Prehash_Entity','Compass Advisor','Readout_Date','Readout Delivery Status','Items Recommended','Entity_to_hash','HashedReadout']]

,Request ID,Create Date,Customer Name,GUID,Partner Name,Prehash_Entity,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Entity_to_hash,HashedReadout
4439,111999,2022-02-08 23:18:47.963,WA-COUNTY OF KING,36324,None,WA36324,Mark Vodka,2022-03-04,NaN,"BCS,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,ST,SSPT","WA36324_MARK VODKA_2022-03-04 00:00:00_BCS,DNA...",150b7a0c6d1a9a8840195514184a09a0
4995,112084,2022-02-15 15:30:57.444,INTERNATIONAL FLAVORS AND FRAGRANCES,10636,None,IN10636,Term,NaT,Complete,"SNTC,SSPT,EC-HTOM,EC-KT,ST,BCS","IN10636_TERM_NaT_SNTC,SSPT,EC-HTOM,EC-KT,ST,BCS",e1e67bc9cfdcc04ad61302ccd9da79d2
5363,112993,2022-02-27 18:06:35.629,"ANTEL URUGUAY,ANTEL URUGUAY ANCEL ANTEL ADMINI...","4548987,37398468",None,AN4548987,Term,NaT,NaN,"BCS,BRW,DNA,EC-KT,PR,SNTC,SSPT,ST","AN4548987_TERM_NaT_BCS,BRW,DNA,EC-KT,PR,SNTC,S...",2a410c1578ea36a88ae1da92ac9ed44d
5364,112994,2022-02-27 18:06:38.609,BANCO DE CORDOBA BANCO DE CORDOBA SA BANCO DE ...,4549041,None,BA4549041,Term,NaT,NaN,"BCS,BRW,EC-AM,EC-HTOM,IRR,EC-KT,PR,SNTC,SSPT,ST","BA4549041_TERM_NaT_BCS,BRW,EC-AM,EC-HTOM,IRR,E...",ded1a477ea302a0cce12773548cfa940
5365,112995,2022-02-27 18:06:41.082,BANCO DE LA CIUDAD DE BUENOS AIRES FUNDACION B...,4549017,None,BA4549017,Term,NaT,NaN,"BCS,BRW,EC-KT,SNTC,SSPT,ST","BA4549017_TERM_NaT_BCS,BRW,EC-KT,SNTC,SSPT,ST",04b4f70916006e2d62d19c4f45dff67e
5373,112991,2022-02-27 18:06:29.805,EMPRESA ARGENTINA DE SOLUCIONES SATELITALES SA...,"162532705,274993982,317887260,62680155,144364432",None,EM162532705,Term,NaT,NaN,"BCS,BRW,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SSPT,ST","EM162532705_TERM_NaT_BCS,BRW,EC-AM,EC-HTOM,EC-...",942bb1e3e8f3cd5b6a897b4de5a83830
5374,112992,2022-02-27 18:06:32.875,BANCO DE GALICIA Y BUENOS AIRES,"179950255,4645125",None,BA179950255,Term,NaT,NaN,"BCS,BRW,EC-HTOM,EC-AM,EC-KT,IRR,PR,SNTC,SSPT,ST","BA179950255_TERM_NaT_BCS,BRW,EC-HTOM,EC-AM,EC-...",59cd6822feed07c1eab407be0cea1f80


In [207]:
# prehash_check = df.loc[(df['Prehash_Entity'].notnull()) & (df['RR (internal)']=='YES') & (df['Campaign Type']=='PROACTIVE') & (df['Lvl1']=='EMEAR-REGION') & (df['Request FY']=='2021')].groupby(['Prehash_Entity','HashedReadout'])[['Request ID']].nunique()

In [208]:
# prehash_check.reset_index(inplace=True)

In [209]:
# prehash_check

In [210]:
# prehash_check.to_excel(r'C:\Users\phsheari\Desktop\CheckData.xlsx', index=False)

In [211]:
df.loc[df['HashedReadout'].notnull()][['Request ID','Customer Name','Entity_to_hash','HashedReadout','Compass Advisor','Readout_Date','Readout Delivery Status','Items Recommended','Campaign Type','RR_INTERNAL']]

,Request ID,Customer Name,Entity_to_hash,HashedReadout,Compass Advisor,Readout_Date,Readout Delivery Status,Items Recommended,Campaign Type,RR_INTERNAL
1,111949,TRAFIKVERKET,TR179365505_VIRIATO EVANGELISTA_2022-02-09 00:...,f0aaa98058ee49a8ec07b2a85c0a5736,Viriato Evangelista,2022-02-09,Complete,SNTC,FIELD,YES
33,111333,KEURIG GREEN MOUNTAIN COFFEE ROASTERS INC,KE212964998_CRISTIAN HERRERA_2022-02-22 00:00:...,75ef3002862e107f66e9add00fb55a43,Cristian Herrera,2022-02-22,Complete,SNTC,TELEMETRY,YES
39,111236,THE FRED W ALBRECHT GROCERY COMPANY,FR2637570_GLORIA CANALES_2022-02-28 00:00:00_SNTC,2091d0524ee0bd5bffe0755fcf15dfc7,Gloria Canales,2022-02-28,Complete,SNTC,TELEMETRY,YES
40,111181,GIVAUDAN FLAVORS CORPORATION,GI123467_GLORIA CANALES_2022-02-28 00:00:00_SNTC,c460162b4a36ecf4b12e70daa98c23ed,Gloria Canales,2022-02-28,Complete,SNTC,TELEMETRY,YES
41,111126,RYAN SPECIALTY GROUP,RY176821284_GLORIA CANALES_2022-02-28 00:00:00...,ff73c3631e4b63fa902f39b78375e521,Gloria Canales,2022-02-28,Complete,SNTC,TELEMETRY,YES
...,...,...,...,...,...,...,...,...,...,...
37371,107566,SYNCRO CORPORATION,SY10368781_SANGEETHA SAMPATH_2021-12-14 00:00:...,8b7b0aebae31b2a3130b9a1251b603cd,Sangeetha Sampath,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",FIELD,YES
37372,107567,ENCOMPASS HEALTH CORPORATION,EN320903967_SANGEETHA SAMPATH_2021-12-14 00:00...,dcbfcc3664ab6063160cf23817ae8636,Sangeetha Sampath,2021-12-14,Complete,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",FIELD,YES
37373,107568,SLIDELL MEMORIAL HOSPITAL,SL3773236_SANGEETHA SAMPATH_2021-12-14 00:00:0...,e51a7718bad6cd5bf5a2caad877af536,Sangeetha Sampath,2021-12-14,Complete,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,YES
37374,107569,MILLIPORE SIGMA,MI2134315_SANGEETHA SAMPATH_2021-12-14 00:00:0...,23880ed49b4c61890731eee5eac8dfed,Sangeetha Sampath,2021-12-14,Complete,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",FIELD,YES


In [212]:
df['Partner Deal'] = df['Deal ID'].apply(lambda x: 'TRUE' if x == -44444 else 'FALSE')

In [213]:
df.loc[df['Deal ID']!=-44444,['Customer Name','Deal ID','Partner Deal','Partner Name']]

,Customer Name,Deal ID,Partner Deal,Partner Name
0,ENBRIDGE INC,55694286,FALSE,None
1,TRAFIKVERKET,NaN,FALSE,None
2,CENTRAL GOVERNMENT UK,NaN,FALSE,None
3,FRIEDE SPRINGER GMBH & CO. KG,NaN,FALSE,None
4,GLOBAL MOBILITY HOLDING BV,NaN,FALSE,None
...,...,...,...,...
37371,SYNCRO CORPORATION,NaN,FALSE,None
37372,ENCOMPASS HEALTH CORPORATION,NaN,FALSE,None
37373,SLIDELL MEMORIAL HOSPITAL,NaN,FALSE,None
37374,MILLIPORE SIGMA,NaN,FALSE,None


In [214]:
### Run through the BDM field as if it were GUIDs, replacing "/" with "," as separators, then expanding lists; 
###   ...replacing emails with Proper Names, then feeding proper Names as lookups to return the Report Group field values. 
###  ... Accounting for older, pre-2021 values will have to be added to the roll-up list

In [215]:
advisor_path = r'C:\Users\phsheari\Documents\Compass Advisors\Advisor Email Translation.xlsx'
advisors_df = pd.read_excel(advisor_path)
advisors_df['cecid'] = advisors_df['cecid'].apply(lambda x: x if pd.notnull(x) else pd.NA)
advisors_df['email'] = advisors_df['cecid'].apply(lambda x: x + '@cisco.com' if pd.notnull(x) else pd.NA)

In [216]:
# advisors_df.loc[advisors_df['name']=='Farha Diba']

In [217]:
advisors = dict(zip(advisors_df['name'],advisors_df['report_group']))
advisors_email = dict(zip(advisors_df['email'],advisors_df['report_group']))
advisors_names = dict(zip(advisors_df['name'], advisors_df['report_group']))

In [218]:
def get_bdm_report_group(bdm):    #"bdm" is the compass advisor column from the tracker
    if pd.isnull(bdm):
        return pd.NA
    elif ',' in bdm or '/' in bdm:
        bdm = bdm.lower().strip().replace('/',',').strip()
        bdmlist = bdm.split(',')
        bdm = bdmlist[0]
    
    if bdm.lower().endswith('@cisco.com'):
        return advisors_email.get(bdm, 'Other')
    else:
        return advisors.get(bdm, advisors_names.get(bdm, 'Other'))    

In [219]:
df['Report Group'] = df['Compass Advisor'].apply(get_bdm_report_group)

In [220]:
# df.loc[df['Compass Advisor'].str.contains('Farha Diba', na=False, case=False)][['Request ID','Create Date','Customer Name','Compass Advisor','Report Group']]

In [221]:
df.groupby(df['Report Group'])['Report Group'].count()

Report Group
Atif Ahmad         2633
Jenee St John      2174
Other              1179
Stacey Edwards    10550
Name: Report Group, dtype: int64

In [222]:
# df.groupby(['Compass Advisor','Report Group'])['Compass Advisor','Report Group'].count()

In [223]:
# df.loc[:,df.columns.str.startswith('O')].columns      #['Projected Booking ($,000)','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year','Opportunity Name','Forecast Stage','Forecast Status']

In [224]:
#forecast_columns_to_drop = ['Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year','Forecast Stage','Forecast Status']  #'Projected Booking ($,000)',

In [225]:
#df.drop(columns=forecast_columns_to_drop, inplace=True)

In [226]:
PIPELINE_COLUMNS = ['Request ID', 'Forecast Status', 'Forecast Stage','Service Source', 'Fiscal Month', 'Fiscal Quarter', 'Fiscal Year', 'Fiscal Y-Q', 'Opportunity Value',]

In [227]:
pipeline = pd.read_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\TrkrPipeline.xlsx', usecols=PIPELINE_COLUMNS)

In [228]:
pipeline.columns

Index(['Request ID', 'Service Source', 'Forecast Stage', 'Forecast Status',
       'Fiscal Month', 'Fiscal Quarter', 'Fiscal Year', 'Fiscal Y-Q',
       'Opportunity Value'],
      dtype='object')

In [229]:
pipeline = pipeline.loc[pipeline['Service Source']=='New'].groupby(['Request ID','Forecast Stage','Forecast Status','Fiscal Month','Fiscal Quarter','Fiscal Year'])[['Opportunity Value']].sum()

In [230]:
pipeline.reset_index(inplace=True)

In [231]:
pipeline['Opportunity Value'] = pipeline['Opportunity Value'].astype(float)

In [232]:
pipeline['Request ID'] = pipeline['Request ID'].astype(str)

In [233]:
pipeline_rename_fields = {'Fiscal Month':'Target Fiscal Month','Fiscal Quarter':'Target Fiscal Quarter','Fiscal Year':'Target Fiscal Year',}

In [234]:
pipeline.rename(columns=pipeline_rename_fields, inplace=True)

In [235]:
replace_na_values = {'Forecast Stage':'','Forecast Status':'','Target Fiscal Month':'','Target Fiscal Quarter':'','Target Fiscal Year':'', 'Opportunity Value':0.0}

In [236]:
df['Request ID'] = df['Request ID'].astype(str)

In [237]:
df = df.merge(pipeline, how='left', on='Request ID')

In [238]:
df.fillna(value=replace_na_values, inplace=True)

In [239]:
df.rename(columns=pipeline_rename_fields, inplace=True)

In [240]:
df.loc[df['Request ID'].isin(finbi_requests), 'Forecast Stage'] = '5 - Closed Won'
df.loc[df['Request ID'].isin(finbi_requests), 'Forecast Status'] = 'Booked'
df.loc[df['Request ID'].isin(finbi_requests), 'WORKING STATUS'] = 'BOOKED'

In [241]:
df.drop(df.loc[df['Request ID']=='undefined'].index, inplace=True)

In [242]:
df['WORKING STATUS'].unique()

array(['SCHEDULED', 'READOUT DONE', 'COMPASS UNDELIVERABLE', 'RR_SET_NO',
       'DECLINED', 'READOUT POSTPONED', 'UNASSIGNED', 'ASSIGNED',
       'EXCEPTION_REVIEW', 'BOOKED', 'READOUT PASSED - RESULT PENDING'],
      dtype=object)

In [243]:
# set_as_date(df, ['Create Date','Date Completed','Readout_Date','Last Updated'])
df['LAST_UPDATED']

0        2022-03-03 18:03 PM Eastern
1        2022-03-03 18:03 PM Eastern
2        2022-03-03 18:03 PM Eastern
3        2022-03-03 18:03 PM Eastern
4        2022-03-03 18:03 PM Eastern
                    ...             
37310    2022-03-03 18:03 PM Eastern
37311    2022-03-03 18:03 PM Eastern
37312    2022-03-03 18:03 PM Eastern
37313    2022-03-03 18:03 PM Eastern
37314    2022-03-03 18:03 PM Eastern
Name: LAST_UPDATED, Length: 37315, dtype: object

In [244]:
df.loc[:,'Create Date'] = df.loc[df['Create Date'].notnull()]['Create Date'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df.loc[df['Date Completed'].notnull(),'Date Completed'] = df.loc[df['Date Completed'].notnull()]['Date Completed'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df.loc[df['Readout_Date'].notnull(),'Readout_Date'] = df.loc[df['Readout_Date'].notnull()]['Readout_Date'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
df.loc[df['Modified'].notnull(),'Modified'] = df.loc[df['Modified'].notnull()]['Modified'].apply(lambda x: x.strftime('%Y-%m-%d')).apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
#df.loc[:,'LAST_UPDATED'] = df.loc[:,'LAST_UPDATED'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M %p') + ' Eastern/NYC') #.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M %p'))

In [245]:
df.loc[df['Sales Level 1'] == 'APJC__','Sales Level 1'] = 'APJC'

In [246]:
df[['Request ID','Create Date','Date Completed','Readout_Date','Modified','LAST_UPDATED']]

,Request ID,Create Date,Date Completed,Readout_Date,Modified,LAST_UPDATED
0,112652,2022-02-17,2022-02-24,2022-03-07,2022-03-03,2022-03-03 18:03 PM Eastern
1,111949,2022-02-07,2022-02-09,2022-02-09,2022-02-16,2022-03-03 18:03 PM Eastern
2,111948,2022-02-07,2022-02-09,2022-03-07,2022-03-01,2022-03-03 18:03 PM Eastern
3,111647,2022-01-28,2022-02-04,NaT,2022-02-03,2022-03-03 18:03 PM Eastern
4,111648,2022-01-28,2022-02-04,NaT,2022-02-03,2022-03-03 18:03 PM Eastern
...,...,...,...,...,...,...
37310,107566,2021-11-24,2021-12-06,2021-12-14,2022-03-01,2022-03-03 18:03 PM Eastern
37311,107567,2021-11-24,2021-12-06,2021-12-14,2022-03-01,2022-03-03 18:03 PM Eastern
37312,107568,2021-11-24,2021-12-06,2021-12-14,2022-03-01,2022-03-03 18:03 PM Eastern
37313,107569,2021-11-24,2021-12-06,2021-12-14,2022-03-01,2022-03-03 18:03 PM Eastern


In [247]:
def item_recommended_count(item_name, items_recommended):#, items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif item_name in items_recommended:
        return 1
    else:
        return 0

In [248]:
df['RRT_BCS']  = df['Items Recommended'].apply(lambda x: item_recommended_count('BCS', x))
df['RRT_EC']   = df['Items Recommended'].apply(lambda x: item_recommended_count('EC', x))
df['RRT_IRR']  = df['Items Recommended'].apply(lambda x: item_recommended_count('IRR', x))
df['RRT_DNAC'] = df['Items Recommended'].apply(lambda x: item_recommended_count('DNA', x))
df['RRT_SSPT'] = df['Items Recommended'].apply(lambda x: item_recommended_count('SSPT', x))
df['RRT_ST']   = df['Items Recommended'].apply(lambda x: item_recommended_count('ST', x))
df['RRT_SNTC'] = df['Items Recommended'].apply(lambda x: item_recommended_count('SNTC', x))
df['RRT_PR']   = df['Items Recommended'].apply(lambda x: item_recommended_count('PR', x))
df['RRT_BRW']  = df['Items Recommended'].apply(lambda x: item_recommended_count('BRW', x))
df['RRT_NULL'] = 0
df['RRT_VBDM'] = 0
df['RRT_LPM+'] = 0

In [249]:
df.loc[df['Items Recommended'].isnull(), 'RRT_NULL'] = 1

In [250]:
df.loc[(df['NEXT_STEPS'].str.contains('engage vbdm', na=False, case=False)) | (df['VBDM'].str.contains('yes', na=False, case=False)), 'RRT_VBDM']=1

In [251]:
df.loc[df['NEXT_STEPS'].str.contains('engage lpm+', na=False, case=False), 'RRT_LPM+']=1

In [252]:
df['LPM_OFFERS'] = df['LPM_OFFER_CREATED'].apply(lambda x: 1 if x == 'Yes' else 0)

In [253]:
df.loc[(df['Readout_Date'].notnull()) & (df['Items Recommended'].notnull()) & (df['LPM_OFFER_CREATED'].notnull())][['Readout_Date','Items Recommended','VBDM','RRT_BCS','RRT_EC','RRT_IRR','RRT_DNAC','RRT_ST','RRT_SSPT','RRT_SNTC','RRT_PR','RRT_BRW','RRT_VBDM','RRT_LPM+','RRT_NULL','LPM_OFFER_CREATED','LPM_OFFERS']]

,Readout_Date,Items Recommended,VBDM,RRT_BCS,RRT_EC,RRT_IRR,RRT_DNAC,RRT_ST,RRT_SSPT,RRT_SNTC,RRT_PR,RRT_BRW,RRT_VBDM,RRT_LPM+,RRT_NULL,LPM_OFFER_CREATED,LPM_OFFERS
51,2022-02-16,"BCS,DNA,BRW,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",NaN,1,1,1,1,1,1,1,1,1,0,1,0,Yes,1
52,2022-02-04,"BCS,BRW,DNA,EC-AM,EC-HTOM,EC-KT,IRR,PR,SNTC,SS...",NaN,1,1,1,1,1,1,1,1,1,0,1,0,Yes,1
61,2022-02-14,"SNTC,SSPT",NaN,0,0,0,0,0,1,1,0,0,0,1,0,Yes,1
69,2022-02-03,SNTC,NaN,0,0,0,0,0,0,1,0,0,0,1,0,Yes,1
164,2022-01-21,"BCS,DNA,EC-AM,EC-KT,IRR,PR,SNTC,SSPT,ST",NaN,1,1,1,1,1,1,1,1,0,0,1,0,Yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37310,2021-12-14,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT",NO,1,1,1,1,0,1,1,0,1,0,1,0,Yes,1
37311,2021-12-14,"BCS,BRW,IRR,EC-HTOM,SNTC,SSPT",NO,1,1,1,0,0,1,1,0,1,0,1,0,Yes,1
37312,2021-12-14,"BCS,EC-HTOM,IRR,SNTC,SSPT,ST",NO,1,1,1,0,1,1,1,0,0,0,1,0,Yes,1
37313,2021-12-14,"BCS,BRW,DNA,EC-HTOM,IRR,SNTC,SSPT,ST",NO,1,1,1,1,1,1,1,0,1,0,1,0,Yes,1


In [254]:
df['Method'] = None

In [255]:
df['Measure Group'] = 'Operations'

In [256]:
df.loc[df['Campaign Type'].str.contains('coverage at renewal', na=False, case=False),'Method'] = 'M6 (Coverage at Renewal)' #[['Request ID','Create Date','Customer Name','Compass Campaign Name','Campaign Type']]

In [257]:
df.loc[(df['Campaign'] == 'PROACTIVE - TAC LEAD') | (df['Campaign Type'].str.contains('test', na=False, case=False)),'Method'] = 'OMIT'

In [258]:
df.loc[df['Campaign Type'].str.contains('field', na=False, case=False),'Method'] = 'FIELD'

In [259]:
df.loc[df['Campaign Type'].str.contains('targeting', na=False, case=False),'Method'] = 'TARGETING'

In [260]:
df.loc[df['Campaign Type'].str.contains('cxib', na=False, case=False),'Method'] = 'M6 (CXIB)'

In [261]:
df.loc[(df['Campaign Type'].str.contains('missed attach', na=False, case=False)) | 
       (df['Campaign Type'].str.contains('coverage program', na=False, case=False)) | 
       (df['Campaign Type'].str.contains('telemetry', na=False, case=False)) |
       (df['Campaign Type'] == 'TAC') |
       (df['Campaign'] == 'REACTIVE - ATTACH REPORT') |
       (df['Campaign'] == 'COVERAGE PROGRAM - FIELD REQUEST') |
       (df['Campaign'] == 'COVERAGE PROGRAM - SNTC PROACTIVE ATTACH') |
       (df['Campaign Type'].str.contains('proactive coverage', na=False, case=False)),'Method'] = 'M1 (Coverage Program)'

In [262]:
df.loc[(df['Campaign Type'].str.contains('proactive', na=False, case=False)) & (df['Campaign'].str.contains('attach', na=False, case=False))][['Request Type','Create Date','Request Type', 'Campaign Type','Campaign','Request FY','Request FQ','Request FM']]

,Request Type,Create Date,Request Type,Campaign Type,Campaign,Request FY,Request FQ,Request FM
4476,Targeting Campaign,2022-02-16,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,3,07
4477,Targeting Campaign,2022-02-16,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,3,07
4478,Targeting Campaign,2022-02-16,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,3,07
4479,Targeting Campaign,2022-02-16,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,3,07
4480,Targeting Campaign,2022-02-16,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,3,07
...,...,...,...,...,...,...,...,...
36355,Targeting Campaign,2021-11-23,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,2,04
36356,Targeting Campaign,2021-11-23,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,2,04
36357,Targeting Campaign,2021-11-23,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,2,04
36358,Targeting Campaign,2021-11-23,Targeting Campaign,PROACTIVE COVERAGE,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,2022,2,04


In [263]:
df['Campaign Type'].unique()

array(['FIELD', 'TARGETING', 'TELEMETRY', 'COVERAGE AT RENEWAL', 'CXIB',
       'TAC', 'PROACTIVE COVERAGE', 'PROACTIVE_OLD', 'NONREPORTING/TESTS'],
      dtype=object)

In [264]:
df['Targeting Campaign Type'].unique()

array(['NON-TARGETING', 'BCS', 'SNTC', 'SUCCESSTRACKS', 'PARTNER', 'SSPT',
       'MAJORS'], dtype=object)

In [265]:
df.loc[(df['Campaign Type']=='COVERAGE PROGRAM') & (df['Campaign']=='COVERAGE PROGRAM - PROACTIVE ATTACH'), 'Campaign'] = 'COVERAGE - SNTC PROACTIVE ATTACH'

In [266]:
def set_offer_focus(campaign_type, target_campaign_type):
    if campaign_type == 'FIELD':
        return 'ANY'
    elif campaign_type == 'CXIB':
        return 'SNTC'
    elif campaign_type == 'COVERAGE AT RENEWAL':
        return 'SNTC'
    elif (campaign_type == 'MISSED ATTACH') or (campaign_type == 'COVERAGE PROGRAM') or (campaign_type == 'PROACTIVE COVERAGE') or (campaign_type == 'TAC') or (campaign_type == 'TELEMETRY'):
        return 'SNTC'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'MAJORS':
        return 'BCS'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'BCS':
        return 'BCS'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'SUCCESSTRACKS':
        return 'ST'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'SSPT':
        return 'SSPT'
    elif campaign_type == 'TARGETING' and target_campaign_type == 'SNTC':
        return 'SNTC'
    else:
        return None

In [267]:
df['Offer Focus'] = df.apply(lambda x: set_offer_focus(x['Campaign Type'], x['Targeting Campaign Type']), axis=1)

In [268]:
df['Value Type'] = 'Actual'

In [269]:
# df.loc[:,df.columns.str.contains('readout', na=False, case=False)]

In [270]:
def apply_fiscal_year(readout_fy, request_fy):
    if isinstance(readout_fy,str):
        if readout_fy.isdigit():
            return readout_fy
    else:
        return request_fy

In [271]:
def apply_fiscal_qtr(readout_fq, request_fq):
    if isinstance(readout_fq,str):
        if readout_fq.isdigit():
            return 'Q' + readout_fq
    else:
        return 'Q' + request_fq

In [272]:
def apply_injection_month(request_fy, request_fm):
    if isinstance(request_fy,str):
        if request_fy.isdigit():
            return ''.join([request_fy,request_fm])
    else:
        return None

In [273]:
def apply_injection_quarter(request_fy, request_fq):
    if isinstance(request_fy,str):
        if request_fy.isdigit():
            return ''.join([request_fy,'-Q',request_fq])
    else:
        return None

In [274]:
# print(list(df.loc[df['Readout FY']]['Readout FY'].unique()))

In [275]:
df['Report FY'] = df.apply(lambda x: apply_fiscal_year(x['Readout FY'], x['Request FY']), axis=1)

In [276]:
df['Report FQ'] = df.apply(lambda x: apply_fiscal_qtr(x['Readout FQ'], x['Request FQ']), axis=1)

In [277]:
df['Injection Month'] = df.apply(lambda x: apply_injection_month(x['Request FY'],x['Request FM']), axis=1)

In [278]:
df['Injection Quarter'] = df.apply(lambda x: apply_injection_quarter(x['Request FY'],x['Request FQ']), axis=1)

In [279]:
df.loc[(df['Campaign'].str.contains('TAC')) ][['Campaign','Campaign Type','Method']].value_counts()

Campaign                                          Campaign Type       Method               
COVERAGE PROGRAM - SNTC PROACTIVE ATTACH          PROACTIVE COVERAGE  M1 (Coverage Program)    861
SUCCESS TRACKS FOR CAMPUS TARGETING - DCN ATTACH  TARGETING           TARGETING                430
COVERAGE PROGRAM - SNTC TAC LEADS                 TAC                 M1 (Coverage Program)    113
PROACTIVE - TAC LEAD                              PROACTIVE_OLD       OMIT                     100
dtype: int64

In [280]:
df.loc[(df['Report FY']=='2022') & (df['Report FQ']=='Q2') & (df['RR_INTERNAL']=='YES') & (df['ALERT_BELOW_25'] == 0) & (df['Method']=='M1 (Coverage Program)')][['Method','Campaign Type','Campaign',]].value_counts()

Method                 Campaign Type       Campaign                                
M1 (Coverage Program)  TELEMETRY           COVERAGE PROGRAM - TELEMETRY                659
                       PROACTIVE COVERAGE  COVERAGE PROGRAM - SNTC PROACTIVE ATTACH    224
                       TAC                 COVERAGE PROGRAM - SNTC TAC LEADS            11
dtype: int64

In [281]:
df.loc[(df['Method']=='FIELD') & (df['Report FY']=='2022')]

,Request ID,Create Date,Created By,Request Type,Modified,Modified By,Date Completed,TARGET_REQUEST_ID,LAST_UPDATED,QA Approve,...,RRT_LPM+,LPM_OFFERS,Method,Measure Group,Offer Focus,Value Type,Report FY,Report FQ,Injection Month,Injection Quarter
0,112652,2022-02-17,gschribe@cisco.com,Reactive,2022-03-03,sansampa@cisco.com,2022-02-24,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202207,2022-Q3
1,111949,2022-02-07,ngulahme@cisco.com,Reactive,2022-02-16,vevangel@cisco.com,2022-02-09,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202207,2022-Q3
2,111948,2022-02-07,macoup@cisco.com,Reactive,2022-03-01,macoup@cisco.com,2022-02-09,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202207,2022-Q3
154,109242,2021-12-17,chrihoff@cisco.com,Reactive,2022-01-28,ebaird@cisco.com,2022-01-10,NaN,2022-03-03 18:03 PM Eastern,False,...,1,0,FIELD,Operations,ANY,Actual,2022,Q2,202205,2022-Q2
155,110407,2022-01-11,vikantha@cisco.com,Reactive,2022-03-01,abierste@cisco.com,2022-01-17,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202206,2022-Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37310,107566,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,FIELD,Operations,ANY,Actual,2022,Q2,202204,2022-Q2
37311,107567,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,FIELD,Operations,ANY,Actual,2022,Q2,202204,2022-Q2
37312,107568,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,FIELD,Operations,ANY,Actual,2022,Q2,202204,2022-Q2
37313,107569,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,FIELD,Operations,ANY,Actual,2022,Q2,202204,2022-Q2


In [282]:
df.loc[(df['Report FY']=='2022') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Campaign Type'].isin(['MISSED ATTACH','COVERAGE PROGRAM','COVERAGE AT RENEWAL']))][['Report FY','Request ID','Create Date','Readout_Date','RR_INTERNAL','Compass Advisor','Deal ID','Method','Campaign Type','Campaign','Customer Name','Readout Delivery Status','OPPTY_STG1_CREATED','LPM_READOUTS_DONE','LPM_READOUTS_DECLINED','OFFER_CREATED']]

,Report FY,Request ID,Create Date,Readout_Date,RR_INTERNAL,Compass Advisor,Deal ID,Method,Campaign Type,Campaign,Customer Name,Readout Delivery Status,OPPTY_STG1_CREATED,LPM_READOUTS_DONE,LPM_READOUTS_DECLINED,OFFER_CREATED
2207,2022,109902,2022-01-11,2022-02-09,YES,Viriato Evangelista,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,BRIGHTLANDS CHEMELOT CAMPUS,Complete,1,0,0,0
2223,2022,109904,2022-01-11,2022-01-25,YES,Yesenia Diaz,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,WILKINSON GROUP PLC,Complete,1,1,0,1
2224,2022,109905,2022-01-11,2022-01-27,YES,Yesenia Diaz,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,BERKLEY GROUP,Complete,1,0,0,0
2225,2022,109909,2022-01-11,2022-01-27,YES,Yesenia Diaz,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,THE FOOTBALL ASSOCIATION,Complete,1,0,0,0
2226,2022,109917,2022-01-11,2022-02-04,YES,Yesenia Diaz,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,AKQUINET AG,Complete,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37148,2022,107258,2021-11-23,2021-12-14,YES,Sangeetha Sampath,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,CENTRAL BANCOMPANY INC,Complete,1,0,0,1
37149,2022,107321,2021-11-23,2021-12-14,YES,Mariangela Pineda,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,BRUCE POWER LP,Complete,1,1,0,1
37150,2022,107203,2021-11-23,2021-12-14,YES,Sangeetha Sampath,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,VISTEON,Complete,1,0,1,0
37151,2022,107317,2021-11-23,2021-12-14,YES,Sangeetha Sampath,NaN,M6 (Coverage at Renewal),COVERAGE AT RENEWAL,COVER THE UNCOVERED - COVERAGE AT RENEWAL,A123 SYSTEMS INC,Complete,1,0,1,0


In [283]:
df.loc[(df['Campaign Type'].str.contains('field', na=False, case=False) | df['Campaign Type'].str.contains('target', na=False, case=False))  & (df['LPM_PLUS'].str.contains('yes', na=False, case=False) | df['NEXT_STEPS'].str.contains('engage lpm', na=False, case=False)),'Method'] = \
df.loc[(df['Campaign Type'].str.contains('field', na=False, case=False) | df['Campaign Type'].str.contains('target', na=False, case=False))  & (df['LPM_PLUS'].str.contains('yes', na=False, case=False) | df['NEXT_STEPS'].str.contains('engage lpm', na=False, case=False)),'Method'] = 'M1 (Coverage Program)'

In [284]:
df.loc[(df['Report FY']=='2022') & (df['Report FQ']=='Q2') & (df['RR_INTERNAL']=='YES') & (df['Sales Level 1'] == 'AMERICAS') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Method'].str.contains('M1', na=False, case=False)) & (df['Campaign Type'].isin(['FIELD','COVERAGE PROGRAM','PROACTIVE COVERAGE','TAC','TELEMETRY','TARGETING'])) & (df['ALERT_BELOW_25'] == 0)]['Campaign Type'].value_counts() #[['Report FY','Request ID','Create Date','Readout_Date','RR_INTERNAL','Compass Advisor','Deal ID','Method','Campaign Type','Campaign','Customer Name','Readout Delivery Status','OPPTY_STG1_CREATED','LPM_READOUTS_DONE','LPM_READOUTS_DECLINED','OFFER_CREATED']]

TELEMETRY             641
FIELD                 307
TARGETING              26
PROACTIVE COVERAGE     22
TAC                    11
Name: Campaign Type, dtype: int64

In [285]:
df.loc[(df['Report FY']=='2022') & (df['Report FQ']=='Q2') & (df['ALERT_BELOW_25'] == 0) & (df['RR_INTERNAL']=='YES') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Method'].str.contains('M1', na=False, case=False)) & (df['Campaign Type'].isin(['TARGETING'])) ]['Campaign Type'].value_counts()  #& (df['ALERT_BELOW_25'] == 0) & (df['RR_INTERNAL']=='YES')  & (df['Sales Level 1'] == 'AMERICAS')

TARGETING    40
Name: Campaign Type, dtype: int64

###  ALTER Field and Target requests to M1 Coverage Program Method where LPM Plus = YES

In [286]:
df.loc[(df['Campaign Type'].str.contains('field', na=False, case=False) | df['Campaign Type'].str.contains('target', na=False, case=False)) & (df['LPM_PLUS'].str.contains('yes', na=False, case=False))][['Request ID','Campaign','Campaign Type','Method','LPM_PLUS','NEXT_STEPS']] #  & (df['LPM_PLUS'].str.contains('yes', na=False, case=False))

,Request ID,Campaign,Campaign Type,Method,LPM_PLUS,NEXT_STEPS
162,108777,REACTIVE,FIELD,M1 (Coverage Program),Yes,NaN
163,108776,REACTIVE,FIELD,M1 (Coverage Program),Yes,NaN
165,108775,REACTIVE,FIELD,M1 (Coverage Program),Yes,NaN
213,108116,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage LPM+
234,107605,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage LPM+
...,...,...,...,...,...,...
37310,107566,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage LPM+
37311,107567,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage LPM+
37312,107568,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage LPM+
37313,107569,REACTIVE,FIELD,M1 (Coverage Program),Yes,Engage LPM+


In [287]:
df.rename(columns={'COMPASS_READOUTS_DONE':'READOUTS_DONE'}, inplace=True)

In [288]:
opps_stg1 = df.loc[(df['Report FY']=='2022') & (df['ALERT_BELOW_25'] == 0) & (df['RR_INTERNAL']=='YES') & (df['OPPTY_STG1_CREATED'] == 1) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL'])) ]   #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [289]:
agg_opps = pd.DataFrame(opps_stg1.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED'].sum())

In [290]:
agg_opps.reset_index(inplace=True)

In [291]:
agg_opps['Measure'] = 'Stg1 Opportunity'
agg_opps.rename(columns={'OPPTY_STG1_CREATED':'Value'}, inplace=True)
agg_opps

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
1,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
2,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Stg1 Opportunity
3,FIELD,Operations,REACTIVE,FIELD,2 SISTER POULTRY LTD,EMEAR-REGION,EMEAR-UKI,COMMERCIAL_EAW,COM_EAW_SEL_NORTH,ANY,Actual,2022,Q3,1,Stg1 Opportunity
4,FIELD,Operations,REACTIVE,FIELD,3M,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,GLL_K_3M,ANY,Actual,2022,Q1,1,Stg1 Opportunity
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6107,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YANTAI WANHUA GROUP,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,ST,Actual,2022,Q2,1,Stg1 Opportunity
6108,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YPFB TRANSPORTE,AMERICAS,LATIN AMERICA,M_C_R,PERU_ANDEAN,ST,Actual,2022,Q1,1,Stg1 Opportunity
6109,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZENSAR TECHNOLOGIES LTD,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ST,Actual,2022,Q2,1,Stg1 Opportunity
6110,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZURICH INSURANCE GROUP,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ST,Actual,2022,Q2,1,Stg1 Opportunity


In [292]:
readouts_done = df.loc[(df['Report FY']=='2022') & (df['READOUTS_DONE'] == 1) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL'])) ] #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [293]:
agg_readouts = pd.DataFrame(readouts_done.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['READOUTS_DONE'].sum())

In [294]:
agg_readouts.reset_index(inplace=True)

In [295]:
agg_readouts['Measure'] = 'Readouts Delivered'
agg_readouts.rename(columns={'READOUTS_DONE':'Value'}, inplace=True)
agg_readouts

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
1,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
2,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Readouts Delivered
3,FIELD,Operations,REACTIVE,FIELD,2 SISTER POULTRY LTD,EMEAR-REGION,EMEAR-UKI,COMMERCIAL_EAW,COM_EAW_SEL_NORTH,ANY,Actual,2022,Q3,1,Readouts Delivered
4,FIELD,Operations,REACTIVE,FIELD,A J BELL HOLDINGS LTD,EMEAR-REGION,EMEAR-UKI,COMMERCIAL_EAW,COM_EAW_SEL_NORTH,ANY,Actual,2022,Q3,1,Readouts Delivered
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YAKULT,APJC,ASEAN_AREA,ASEAN_PH,ASEAN_COM_PHL,ST,Actual,2022,Q2,1,Readouts Delivered
3240,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YAMAHA MOTOR CO. LTD.,APJC,JAPAN__,JP_COMMERCIAL_OTH,JP_COM_SELECT,ST,Actual,2022,Q2,1,Readouts Delivered
3241,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YANTAI WANHUA GROUP,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,ST,Actual,2022,Q2,1,Readouts Delivered
3242,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZENSAR TECHNOLOGIES LTD,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ST,Actual,2022,Q2,1,Readouts Delivered


In [296]:
offers_created = df.loc[(df['Report FY']=='2022') & (df['OFFER_CREATED'] == 1) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL'])) ]   #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [297]:
agg_offers = pd.DataFrame(offers_created.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OFFER_CREATED'].sum())

In [298]:
agg_offers.reset_index(inplace=True)

In [299]:
agg_offers['Measure'] = 'Offers Created'
agg_offers.rename(columns={'OFFER_CREATED':'Value'}, inplace=True)
agg_offers

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,1,Offers Created
1,FIELD,Operations,REACTIVE,FIELD,ADVENT HEALTH,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,SOUTHEAST OP,ANY,Actual,2022,Q3,1,Offers Created
2,FIELD,Operations,REACTIVE,FIELD,AFLAC,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,TAG OPERATION,ANY,Actual,2022,Q3,1,Offers Created
3,FIELD,Operations,REACTIVE,FIELD,ALLY_FINANCIAL,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,MIDWEST ENTERPRISE OPERATION,ANY,Actual,2022,Q3,1,Offers Created
4,FIELD,Operations,REACTIVE,FIELD,AMERICAN FINANCIAL GROUP,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,MIDWEST ENTERPRISE OPERATION,ANY,Actual,2022,Q3,1,Offers Created
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
883,TARGETING,Operations,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,COOK GROUP INC,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,CENTRAL EAST COMMERCIAL OPERATION,ST,Actual,2022,Q2,1,Offers Created
884,TARGETING,Operations,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,DEUTSCHER SPARKASSEN-UND GIROVERBAND EV,EMEAR-REGION,EMEAR_GERMANY,ENTERPRISE_I_DEU,ENT_DEU_FSI,ST,Actual,2022,Q3,1,Offers Created
885,TARGETING,Operations,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,THE WORLD BANK GROUP - PRIMARY,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES EAST,ATLANTIC ENTERPRISE OPERATION,ST,Actual,2022,Q2,1,Offers Created
886,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NEW JERSEY INSTITUTE OF TECHNOLOGY,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-NORTHEAST OPERATION,ST,Actual,2022,Q2,1,Offers Created


In [300]:
uncovered_pipeline = df.loc[(df['Report FY']=='2022') & (df['UNCOVERED_OPPTY_VALUE'] > 0) & (df['Campaign Type'].isin(['TAC','TARGETING','TELEMETRY','FIELD','PROACTIVE COVERAGE','COVERAGE AT RENEWAL'])) ]    #& (df['Method'].str.contains('M1', na=False, case=False)) 

In [301]:
agg_unc_pipeline = pd.DataFrame(uncovered_pipeline.groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['UNCOVERED_OPPTY_VALUE'].sum())

In [302]:
agg_unc_pipeline.reset_index(inplace=True)

In [303]:
agg_unc_pipeline['Measure'] = 'Uncovered Opp Value $(000''s)'
agg_unc_pipeline.rename(columns={'UNCOVERED_OPPTY_VALUE':'Value'},inplace=True)
agg_unc_pipeline

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,AFLAC,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,TAG OPERATION,ANY,Actual,2022,Q3,24.0,Uncovered Opp Value $(000s)
1,FIELD,Operations,REACTIVE,FIELD,ASPIRE TECHNOLOGY PARTNERS LLC,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET TERRITORY,ANY,Actual,2022,Q3,0.6,Uncovered Opp Value $(000s)
2,FIELD,Operations,REACTIVE,FIELD,BAKER HUGHES INC,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,GLL_M_BAKER HUGHES,ANY,Actual,2022,Q3,75.0,Uncovered Opp Value $(000s)
3,FIELD,Operations,REACTIVE,FIELD,BANCO SANTANDER,AMERICAS,LATIN AMERICA,M_C_R,ARGENTINA_P_U,ANY,Actual,2022,Q3,0.6,Uncovered Opp Value $(000s)
4,FIELD,Operations,REACTIVE,FIELD,BAYCARE,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,SOUTHEAST OP,ANY,Actual,2022,Q2,342.6,Uncovered Opp Value $(000s)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
791,M6 (Coverage at Renewal),Operations,COVER THE UNCOVERED - COVERAGE AT RENEWAL,COVERAGE AT RENEWAL,ZEUS INTERNATIONAL,AMERICAS,US COMMERCIAL,COMMERCIAL EAST AREA,SOUTH EAST COMMERCIAL OPERATION,SNTC,Actual,2022,Q2,3.0,Uncovered Opp Value $(000s)
792,TARGETING,Operations,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,DEUTSCHER SPARKASSEN-UND GIROVERBAND EV,EMEAR-REGION,EMEAR_GERMANY,ENTERPRISE_I_DEU,ENT_DEU_FSI,ST,Actual,2022,Q3,1.8,Uncovered Opp Value $(000s)
793,TARGETING,Operations,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,THE WORLD BANK GROUP - PRIMARY,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES EAST,ATLANTIC ENTERPRISE OPERATION,ST,Actual,2022,Q2,63.6,Uncovered Opp Value $(000s)
794,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NEW JERSEY INSTITUTE OF TECHNOLOGY,AMERICAS,US PS MARKET SEGMENT,SLED-EAST AREA,SLED-NORTHEAST OPERATION,ST,Actual,2022,Q2,31.2,Uncovered Opp Value $(000s)


In [304]:
group_of_measures = pd.concat([agg_opps,agg_readouts,agg_offers,agg_unc_pipeline], axis=1)

In [305]:
group_of_measures

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,...,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Value,Measure
0,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,TAG OPERATION,ANY,Actual,2022,Q3,24.0,Uncovered Opp Value $(000s)
1,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,AMERICAS,AMERICAS_SP,ASP US SEGMENTS,ASP MID MARKET TERRITORY,ANY,Actual,2022,Q3,0.6,Uncovered Opp Value $(000s)
2,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,...,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,GLL_M_BAKER HUGHES,ANY,Actual,2022,Q3,75.0,Uncovered Opp Value $(000s)
3,FIELD,Operations,REACTIVE,FIELD,2 SISTER POULTRY LTD,EMEAR-REGION,EMEAR-UKI,COMMERCIAL_EAW,COM_EAW_SEL_NORTH,ANY,...,AMERICAS,LATIN AMERICA,M_C_R,ARGENTINA_P_U,ANY,Actual,2022,Q3,0.6,Uncovered Opp Value $(000s)
4,FIELD,Operations,REACTIVE,FIELD,3M,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES CENTRAL,GLL_K_3M,ANY,...,AMERICAS,GLOBAL ENTERPRISE SEGMENT,GES SOUTH,SOUTHEAST OP,ANY,Actual,2022,Q2,342.6,Uncovered Opp Value $(000s)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6107,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YANTAI WANHUA GROUP,APJC,GREATER_CHINA,CN_NORTH,CN_ENT_NORTH_MET,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6108,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,YPFB TRANSPORTE,AMERICAS,LATIN AMERICA,M_C_R,PERU_ANDEAN,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6109,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZENSAR TECHNOLOGIES LTD,APJC,INDIA_AREA,IND_ENTERPRISE_MAJOR,IND_ENT_STRATEGIC,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6110,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZURICH INSURANCE GROUP,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ST,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [306]:
#df.loc[(df['Report FY']=='2022') ].groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED','READOUTS_DONE', 'OFFER_CREATED','UNCOVERED_OPPTY_VALUE'].sum().stack()
agg_rpt = df.loc[(df['Report FY']=='2022') ].groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED','READOUTS_DONE', 'OFFER_CREATED','UNCOVERED_OPPTY_VALUE'].sum().stack()

<ipython-input-306-d1763e190e15>:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  agg_rpt = df.loc[(df['Report FY']=='2022') ].groupby(['Method', 'Measure Group','Campaign', 'Campaign Type','Customer Name', 'Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4','Offer Focus', 'Value Type','Report FY','Report FQ'])['OPPTY_STG1_CREATED','READOUTS_DONE', 'OFFER_CREATED','UNCOVERED_OPPTY_VALUE'].sum().stack()


In [307]:
agg_report = pd.DataFrame(agg_rpt.reset_index()).rename(columns={'level_13':'Measure',0:'Value'})

In [308]:
agg_report.loc[agg_report['Measure']=='UNCOVERED_OPPTY_VALUE']    #.pivot_table(index=['Measure Group','Measure SubGroup', 'Sales Level 1','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value')

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Measure,Value
3,FIELD,Operations,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
7,FIELD,Operations,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
11,FIELD,Operations,REACTIVE,FIELD,(SAA) SOUTH AFRICAN AIRWAYS,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
15,FIELD,Operations,REACTIVE,FIELD,(SAPS) – SOUTH AFRICAN POLICE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
19,FIELD,Operations,REACTIVE,FIELD,(SARS) SOUTH AFRICAN RESERVE SERVICES,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,ANY,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60747,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZIMPLATS HOLDINGS LIMITED (ZIMPLATS),EMEAR-REGION,EMEAR_MEA,SMALL_MEA,CLUSTER_VELOCITY,ST,Actual,2022,Q1,UNCOVERED_OPPTY_VALUE,0.0
60751,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZLM VERZEKERINGEN,EMEAR-REGION,EMEAR-NORTH,COUNTRY_NETHERLANDS,COMMERCIAL_NLD,ST,Actual,2022,Q1,UNCOVERED_OPPTY_VALUE,0.0
60755,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZURICH INSURANCE GROUP,EMEAR-REGION,EMEAR-CENTRAL,COUNTRY_SWITZERLAND,SWITZERLAND_CGEM,ST,Actual,2022,Q2,UNCOVERED_OPPTY_VALUE,0.0
60759,TARGETING,Operations,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,ZWEITES DEUTSCHES FERNSEHEN GEMEINNUTZIGE ANST...,EMEAR-REGION,EMEAR_SP,SP_NORTH,MEDIA_GROUP,ST,Actual,2022,Q3,UNCOVERED_OPPTY_VALUE,0.0


In [309]:
agg_report.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60764 entries, 0 to 60763
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Method         60764 non-null  object 
 1   Measure Group  60764 non-null  object 
 2   Campaign       60764 non-null  object 
 3   Campaign Type  60764 non-null  object 
 4   Customer Name  60764 non-null  object 
 5   Sales Level 1  60764 non-null  object 
 6   Sales Level 2  60764 non-null  object 
 7   Sales Level 3  60764 non-null  object 
 8   Sales Level 4  60764 non-null  object 
 9   Offer Focus    60764 non-null  object 
 10  Value Type     60764 non-null  object 
 11  Report FY      60764 non-null  object 
 12  Report FQ      60764 non-null  object 
 13  Measure        60764 non-null  object 
 14  Value          60764 non-null  float64
dtypes: float64(1), object(14)
memory usage: 7.0+ MB


In [310]:
target_dtypes = {'Report FY': str, 'Value':np.float64}

In [311]:
targets = pd.read_excel(r'C:\Users\phsheari\Documents\Compass SNOWFLAKE\data_download\Mock_Dashboard Targets_SLTest.xlsx', sheet_name = 'Targets', dtype = target_dtypes, na_values = 0.0)

In [312]:
# targets = targets.loc[targets['Measure']!='DATA_PACKAGE_CREATED']

In [313]:
targets.loc[targets['Offer Focus']=='SNTC, PSS', 'Offer Focus'] = 'SNTC'

In [314]:
targets.loc[targets['Value Type']=='Actual']

,Method,Measure Group,Campaign,Campaign Type,Customer Name,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Offer Focus,Value Type,Report FY,Report FQ,Measure,Value
384,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,OPPTY_STG1_CREATED,6.000
385,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,READOUTS_DONE,30.000
386,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,OFFER_CREATED,23.000
387,M1 (Coverage Program),Operations,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,UNCOVERED_OPPTY_VALUE,4776.000
388,M1 (Coverage Program),Bookings-New,COVERAGE PROGRAM - SNTC PROACTIVE ATTACH,PROACTIVE COVERAGE,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q1,Incr ARR $(000s),8000.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,M1 (Coverage Program),Bookings-New,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,NaN,EMEAR-REGION,NaN,NaN,NaN,SNTC,Actual,2022,Q2,Incr ARR $(000s),3.242
794,M1 (Coverage Program),Bookings-New,COVERAGE PROGRAM - FIELD REQUEST,FIELD,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q3,Incr ARR $(000s),1094.407
795,M1 (Coverage Program),Bookings-New,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q3,Incr ARR $(000s),89.966
796,M1 (Coverage Program),Bookings-New,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,NaN,AMERICAS,NaN,NaN,NaN,SNTC,Actual,2022,Q3,Incr ARR $(000s),11.629


In [315]:
targets['Method'].unique()

array(['M1 (Coverage Program)', 'M6 (Coverage at Renewal)'], dtype=object)

In [316]:
opp_flow_dash = pd.concat([agg_report, targets]).fillna(0.0)

In [317]:
opp_flow_dash.loc[(opp_flow_dash['Measure Group'].str.contains('operations', na=False, case=False))].pivot_table(index=['Method','Measure Group', 'Sales Level 1','Sales Level 2','Sales Level 3','Sales Level 4','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)

Report FY                                                                                                   2022  \
Report FQ                                                                                                     Q1   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       1.0   
                                                                                    READOUTS_DONE            1.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                                     ASP LUMEN      OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                          
Value Type                                                                                                Target   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                                     ASP LUMEN      OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                     Q2   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   AS

In [318]:
targets['Measure Group'].unique()

array(['Operations', 'Bookings-New', 'Bookings-Upsell'], dtype=object)

In [319]:
# bookings = finbi_df #.loc[finbi_df['Partner Deal']==False][['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'CAV ID', 'CAV Name',
#                        'GUID', 'GUID Name', 'Partner', 'Booked Deal ID', 'CX Product Category',
#                        'Fiscal Year', 'Fiscal Quarter ID', 'Fiscal Week ID',
#                        'Fiscal Week of Year', 'ReportWeek', 'ProductType', 'TCV', 'ACV',
#                        'Request ID', 'Readout_Date',  'Campaign', 'Campaign Type',
#                        'Target Campaign Flag', 'Partner Deal', 'Service Source',
#                        'Compass GUID Name', 'Compass Deal ID', 'Compass Advisor',
#                        'Readout Delivery Status', 'Items Recommended']]

In [320]:
# bookings['Sales Level 1'] = bookings['Sales Level 1'].str.upper()
# bookings['Sales Level 2'] = bookings['Sales Level 2'].str.upper()
# bookings['Sales Level 3'] = bookings['Sales Level 3'].str.upper()
# bookings['Sales Level 4'] = ''

In [321]:
# bookings

In [322]:
def set_campaign_product_focus(campaign_type, campaign_name, product_type):
    if campaign_type.upper() == 'FIELD' and product_type.upper() == 'SNTC':
        return 'SNTC'
    elif campaign_type.upper() == 'FIELD' and 'BCS' in product_type.upper():
        return 'BCS'
    elif campaign_type.upper() == 'FIELD' and product_type.upper() != 'SNTC' and 'BCS' not in product_type.upper():
        return product_type 
    elif campaign_type.upper() == 'TARGETING':
        if 'BCS' in campaign_name:
            return 'BCS'
        elif 'ST' in campaign_name or 'SUCCESS' in campaign_name:
            return 'ST'
        elif 'SSPT' in campaign_name or 'SOLUTION' in campaign_name:
            return 'SSPT'
        elif 'CTU' in campaign_name:
            return 'SNTC'
    elif (campaign_type.upper() == 'MISSED ATTACH' or campaign_type.upper() == 'COVERAGE PROGRAM'):
        return 'Incr ARR $(000''s)'
    elif campaign_type.upper() == 'COVERAGE AT RENEWAL':
        return 'Incr ARR $(000s)'

In [323]:
def get_lpm_plus_value(request_id):
    #import pdb
    #pdb.set_trace()
    lpm_plus_value = None
    try:
        if df.loc[df['Request ID'] == str(request_id),'LPM_PLUS'].values[0]:
            lpm_plus_value = df.loc[df['Request ID'] == str(request_id),'LPM_PLUS'].values[0]
            if isinstance(lpm_plus_value, float):
                lpm_plus_value = lpm_plus_value.astype(str)
                
    except:
        lpm_plus_value = 'UNKNOWN'
    
    return lpm_plus_value.upper()

In [324]:
def set_method(campaign_type, request_id):
    lpm_plus_value = get_lpm_plus_value(request_id)
    
    if isinstance(lpm_plus_value, str):
        if lpm_plus_value.upper() == 'YES' and campaign_type.upper() == 'COVERAGE AT RENEWAL':
            return 'M6 (Coverage at Renewal)'
    
        elif lpm_plus_value.upper() == 'YES' and campaign_type.upper() != 'COVERAGE AT RENEWAL':
            return 'M1 (Coverage Program)'
    
    elif campaign_type.upper() == 'FIELD':
        return 'FIELD'
    
    elif campaign_type.upper() == 'TARGETING':    
        return 'TARGETING'

In [325]:
df.loc[df['Request ID']=='985','LPM_PLUS']

6435    NaN
Name: LPM_PLUS, dtype: object

In [326]:
get_lpm_plus_value(105803)

'YES'

In [327]:
def set_measure_group(campaign_type, campaign_name, service_source, offer_focus, request_id):
    if request_id:
        lpm_plus_value = get_lpm_plus_value(request_id)
    
    if pd.notnull(lpm_plus_value) or lpm_plus_value != np.nan():
        if pd.isnull(service_source) or service_source == 'NA' or service_source == 'N/A' or service_source.upper() == 'UNKNOWN' or service_source == '':
            del lpm_plus_value
            return 'Bookings-Unknown'
        if campaign_type.upper() == 'FIELD' and service_source.upper() == 'NEW' and lpm_plus_value.upper() != 'YES':
            del lpm_plus_value
            return 'Bookings-New'
        elif campaign_type.upper() == 'TARGETING' and service_source.upper() == 'NEW' and lpm_plus_value.upper() != 'YES':
            del lpm_plus_value
            return 'Bookings-New'
        elif (campaign_type.upper() == 'MISSED ATTACH' or campaign_type.upper() == 'COVERAGE PROGRAM') and offer_focus.upper() == 'SNTC':
            del lpm_plus_value
            return 'Bookings-New'
        elif campaign_type.upper() == 'COVERAGE AT RENEWAL' and offer_focus.upper( ) == 'SNTC':
            del lpm_plus_value
            return 'Bookings-New'
        else:
            return 'Bookings-Upsell'
    elif campaign_type.upper() == 'FIELD' and service_source.upper() == 'NEW':
        return 'Bookings-New'
    elif campaign_type.upper() == 'TARGETING' and service_source.upper() == 'NEW':
        return 'Bookings-New'
    elif (campaign_type.upper() == 'MISSED ATTACH' or campaign_type.upper() == 'COVERAGE PROGRAM') and offer_focus.upper() == 'SNTC':
        return 'Bookings-New'
    elif campaign_type.upper() == 'COVERAGE AT RENEWAL' and offer_focus.upper( ) == 'SNTC':
        return 'Bookings-New'
    else:
        return 'Bookings-Upsell'

In [328]:
# bookings['Method'] = bookings.apply(lambda x: set_method(x['Campaign Type'], x['Request ID']), axis=1)

In [329]:
# bookings.loc[bookings['Campaign Type'].isnull()]

In [330]:
# bookings['Offer Focus'] = bookings.apply(lambda x: set_campaign_product_focus(x['Campaign Type'], x['Campaign'], x['ProductType']), axis=1)

In [331]:
# bookings['Measure'] = 'Incr ARR $(000s)'

In [332]:
# bookings['Measure Group'] = bookings.apply(lambda x: set_measure_group(x['Campaign Type'], x['Campaign'], x['Service Source'], x['Offer Focus'], x['Request ID']), axis=1)

In [333]:
# bookings['Report FY'] = bookings['Fiscal Year'].astype(str)

In [334]:
# bookings['Report FQ'] = [(x[-2:]) for x in bookings['Fiscal Quarter ID']]

In [335]:
# bookings['Value Type'] = 'Actual'

In [336]:
# bookings['Customer Name'] = bookings['CAV Name']

In [337]:
# bookings['Value'] = bookings['ACV']/1000.00

In [338]:
# sub_bookings = bookings.loc[(bookings['Report FY']=='2022')][['Sales Level 1','Sales Level 2', 'Sales Level 3', 'Sales Level 4','Campaign', 'Campaign Type','Report FY','Report FQ', 'Offer Focus','Method','Measure Group','Measure','Value Type','Value']].fillna(0.0) #.pivot_table(index=['Measure Group','Measure SubGroup', 'Sales Level 1','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)

In [339]:
# sub_bookings.shape

In [340]:
# sub_bookings['Measure Group'].unique()

In [341]:
opp_flow_dash[['Sales Level 1', 'Sales Level 2', 'Sales Level 3', 'Sales Level 4', 'Campaign', 'Campaign Type','Customer Name','Report FY', 'Report FQ', 'Offer Focus', 'Method', 'Measure Group', 'Measure', 'Value Type', 'Value']]

,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Campaign,Campaign Type,Customer Name,Report FY,Report FQ,Offer Focus,Method,Measure Group,Measure,Value Type,Value
0,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,OPPTY_STG1_CREATED,Actual,1.000
1,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,READOUTS_DONE,Actual,1.000
2,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,OFFER_CREATED,Actual,0.000
3,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(ACSA) AIRPORTS COMPANY OF SOUTH AFRICA,2022,Q2,ANY,FIELD,Operations,UNCOVERED_OPPTY_VALUE,Actual,0.000
4,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,REACTIVE,FIELD,(PIC) PUBLIC INVESTMENT CORPORATION,2022,Q2,ANY,FIELD,Operations,OPPTY_STG1_CREATED,Actual,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
793,EMEAR-REGION,0.0,0.0,0.0,SUCCESS TRACKS FOR CAMPUS TARGETING - DYNAMIC,TARGETING,0.0,2022,Q2,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,3.242
794,AMERICAS,0.0,0.0,0.0,COVERAGE PROGRAM - FIELD REQUEST,FIELD,0.0,2022,Q3,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,1094.407
795,AMERICAS,0.0,0.0,0.0,SUCCESS TRACKS DYNAMIC CAMPAIGN,TARGETING,0.0,2022,Q3,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,89.966
796,AMERICAS,0.0,0.0,0.0,BCS 3.0 TARGETING GLOBAL COMPASS IDENTIFIED RI...,TARGETING,0.0,2022,Q3,SNTC,M1 (Coverage Program),Bookings-New,Incr ARR $(000s),Actual,11.629


In [342]:
opp_flow_dash.pivot_table(index=['Method','Measure Group', 'Sales Level 1','Sales Level 2','Sales Level 3','Sales Level 4','Measure'], columns=['Report FY','Report FQ','Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)

Report FY                                                                                                   2022  \
Report FQ                                                                                                     Q1   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       1.0   
                                                                                    READOUTS_DONE            1.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                                     ASP LUMEN      OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                          
Value Type                                                                                                Target   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   ASP TELCO MOBILE ASP AMW        OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   
                                                                     ASP LUMEN      OFFER_CREATED            0.0   
...                                                                                                          ...   
TARGETING Operations    EMEAR-REGION  EMEAR_SP      SP_VDF_LG        LIBERTY_GLOBAL UNCOVERED_OPPTY_VALUE    0.0   
                                                                     VODAFONE_      OFFER_CREATED            0.0   
                                                                                    OPPTY_STG1_CREATED       0.0   
                                                                                    READOUTS_DONE            0.0   
                                                                                    UNCOVERED_OPPTY_VALUE    0.0   

Report FY                                                                                                         \
Report FQ                                                                                                     Q2   
Value Type                                                                                                Actual   
Method    Measure Group Sales Level 1 Sales Level 2 Sales Level 3    Sales Level 4  Measure                        
FIELD     Operations    AMERICAS      AMERICAS_SP   AS

In [343]:
# opp_flow_dash = pd.concat([opp_flow_dash, sub_bookings]).fillna(0.0)

In [344]:
opp_flow_dash.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61562 entries, 0 to 797
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Method         61562 non-null  object 
 1   Measure Group  61562 non-null  object 
 2   Campaign       61562 non-null  object 
 3   Campaign Type  61562 non-null  object 
 4   Customer Name  61562 non-null  object 
 5   Sales Level 1  61562 non-null  object 
 6   Sales Level 2  61562 non-null  object 
 7   Sales Level 3  61562 non-null  object 
 8   Sales Level 4  61562 non-null  object 
 9   Offer Focus    61562 non-null  object 
 10  Value Type     61562 non-null  object 
 11  Report FY      61562 non-null  object 
 12  Report FQ      61562 non-null  object 
 13  Measure        61562 non-null  object 
 14  Value          61562 non-null  float64
dtypes: float64(1), object(14)
memory usage: 7.5+ MB


In [345]:
c1 = opp_flow_dash.pivot_table(index=['Method','Measure Group', 'Campaign Type','Campaign','Customer Name','Offer Focus','Sales Level 1','Sales Level 2','Sales Level 3','Sales Level 4','Measure','Report FY','Report FQ',], columns=['Value Type'], values='Value', aggfunc='sum', margins=False).fillna(0.0)  #(opp_flow_dash['Measure SubGroup'].str.contains('operations', na=False, case=False)) & 

In [346]:
c1.reset_index(inplace=True)

In [347]:
c1.loc[c1['Measure Group'].str.contains('operat', na=False, case=False)]['Offer Focus'].unique()

array(['ANY', 'SNTC', 'BCS', 'ST'], dtype=object)

In [348]:
c1.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\Opp_Flow_Dashboard.xlsx', sheet_name = 'OpportunityFlow' , index=False)

In [349]:
c1['Measure'].unique()

array(['OFFER_CREATED', 'OPPTY_STG1_CREATED', 'READOUTS_DONE',
       'UNCOVERED_OPPTY_VALUE', 'Incr ARR $(000s)'], dtype=object)

In [350]:
c1.loc[(c1['Measure']=='READOUTS_DONE') & (c1['Method'].str.contains('M1 \(Coverage')) & (c1['Report FQ']=='Q2')]

Value Type,Method,Measure Group,Campaign Type,Campaign,Customer Name,Offer Focus,Sales Level 1,Sales Level 2,Sales Level 3,Sales Level 4,Measure,Report FY,Report FQ,Actual,Target
9222,M1 (Coverage Program),Operations,FIELD,REACTIVE,(SAPS) – SOUTH AFRICAN POLICE SERVICES,ANY,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,READOUTS_DONE,2022,Q2,1.0,0.0
9226,M1 (Coverage Program),Operations,FIELD,REACTIVE,(SARS) SOUTH AFRICAN RESERVE SERVICES,ANY,EMEAR-REGION,EMEAR_MEA,SMALL_MEA,SMALL_SSA,READOUTS_DONE,2022,Q2,1.0,0.0
9230,M1 (Coverage Program),Operations,FIELD,REACTIVE,ABSA GROUP,ANY,EMEAR-REGION,EMEAR_MEA,CLUSTER_SSA,COUNTRY_ZAF,READOUTS_DONE,2022,Q2,1.0,0.0
9234,M1 (Coverage Program),Operations,FIELD,REACTIVE,ABU DHABI ISLAMIC BANK ADIB,ANY,EMEAR-REGION,EMEAR_MEA,CLUSTER_GULF,COUNTRY_ARE,READOUTS_DONE,2022,Q2,1.0,0.0
9238,M1 (Coverage Program),Operations,FIELD,REACTIVE,ADCB,ANY,EMEAR-REGION,EMEAR_MEA,CLUSTER_GULF,COUNTRY_ARE,READOUTS_DONE,2022,Q2,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24258,M1 (Coverage Program),Operations,TELEMETRY,COVERAGE PROGRAM - TELEMETRY,ZENTIS FOOD SOLUTIONS NORTH AMERICA LLC,SNTC,AMERICAS,US COMMERCIAL,COMMERCIAL CENTRAL AREA,HEARTLAND-GULF COMMERCIAL OPERATION,READOUTS_DONE,2022,Q2,1.0,0.0
24262,M1 (Coverage Program),Operations,TELEMETRY,COVERAGE PROGRAM - TELEMETRY,ZETEC INC,SNTC,AMERICAS,US COMMERCIAL,COMMERCIAL WEST AREA,UNION PACIFIC COMMERCIAL OPERATION,READOUTS_DONE,2022,Q2,0.0,0.0
24266,M1 (Coverage Program),Operations,TELEMETRY,COVERAGE PROGRAM - TELEMETRY,ZIFF BROTHERS,SNTC,AMERICAS,US COMMERCIAL,COMMERCIAL EAST AREA,TRI-STATE COMMERCIAL OPERATION,READOUTS_DONE,2022,Q2,0.0,0.0
24274,M1 (Coverage Program),Operations,TELEMETRY,COVERAGE PROGRAM - TELEMETRY,ZOLL MEDICAL CORPORATION-BMT,SNTC,AMERICAS,US COMMERCIAL,COMMERCIAL EAST AREA,NORTH EAST COMMERCIAL OPERATION,READOUTS_DONE,2022,Q2,0.0,0.0


### Add Measure Group, Measure SubGroup, Offer Focus, to bookings in order to create a table like the agg_report which can be vertically stacked with the Target file data

In [351]:
opp_flow_dash.loc[opp_flow_dash['Value Type']=='Actual']['Measure Group'] #.unique()

0        Operations
1        Operations
2        Operations
3        Operations
4        Operations
           ...     
793    Bookings-New
794    Bookings-New
795    Bookings-New
796    Bookings-New
797    Bookings-New
Name: Measure Group, Length: 61178, dtype: object

In [352]:
# df[['Request ID','Created Date','Customer Name','Campaign','Alert Below 25', 'Requester Name','Requester Function','SSP','Distribution List','RR_internal', 'SAV ID', 
#     'Compass Advisor', 'Customer Identifier','Deal ID','Declined Reason','Email Sent to SSX','Method','Measure Group','Readout Date','Next Steps', 'Items Recommended',
#     'LPM Plus Assigned','LPM Plus Engaged','LPM Plus Scheduled','LPM Plus Unassigned','LPM Readouts Declined','LPM Readouts Done',
#     'Offer Created','Oppty Stg1 Created','Readouts Done','SNTC Estimate','Uncovered Oppty Value','Partner Name','Readout Date',
#     'Readout Delivery Status', 'Report FY','Report FQ','Sales Level 1','Sales Level 2','Sales Level 3', 'Sales Level 4','WORKING STATUS']]

In [353]:
# Done

In [354]:
# ['OPPTY_STG1_CREATED','DATA_PACKAGE_CREATED','READOUTS_DONE','OFFER_CREATED']

In [355]:
df.to_pickle(excel_file_path_sfdc_version)
df.to_excel(excel_file_path_tracker,sheet_name='TrackerData_' + today_file, index=False)
df.to_excel(excel_file_path_tableau,sheet_name='TableauData_' + today_file, index=False)

C:\Users\phsheari\.conda\envs\compass_env\lib\site-packages\xlsxwriter\worksheet.py:986: UserWarning: Ignoring URL 'https://cx-tableau-stage.cisco.com/' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "


In [356]:
finish = dt.datetime.now()
elapsed = (finish - start)
print(elapsed/60.0)

0:00:11.350142


In [357]:
df

,Request ID,Create Date,Created By,Request Type,Modified,Modified By,Date Completed,TARGET_REQUEST_ID,LAST_UPDATED,QA Approve,...,RRT_LPM+,LPM_OFFERS,Method,Measure Group,Offer Focus,Value Type,Report FY,Report FQ,Injection Month,Injection Quarter
0,112652,2022-02-17,gschribe@cisco.com,Reactive,2022-03-03,sansampa@cisco.com,2022-02-24,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202207,2022-Q3
1,111949,2022-02-07,ngulahme@cisco.com,Reactive,2022-02-16,vevangel@cisco.com,2022-02-09,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202207,2022-Q3
2,111948,2022-02-07,macoup@cisco.com,Reactive,2022-03-01,macoup@cisco.com,2022-02-09,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,FIELD,Operations,ANY,Actual,2022,Q3,202207,2022-Q3
3,111647,2022-01-28,vikantha@cisco.com,Research,2022-02-03,navedeva@cisco.com,2022-02-04,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,TARGETING,Operations,BCS,Actual,2022,Q2,202206,2022-Q2
4,111648,2022-01-28,vikantha@cisco.com,Research,2022-02-03,navedeva@cisco.com,2022-02-04,NaN,2022-03-03 18:03 PM Eastern,False,...,0,0,TARGETING,Operations,BCS,Actual,2022,Q2,202206,2022-Q2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37310,107566,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,M1 (Coverage Program),Operations,ANY,Actual,2022,Q2,202204,2022-Q2
37311,107567,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,M1 (Coverage Program),Operations,ANY,Actual,2022,Q2,202204,2022-Q2
37312,107568,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,M1 (Coverage Program),Operations,ANY,Actual,2022,Q2,202204,2022-Q2
37313,107569,2021-11-24,navedeva@cisco.com,Targeting Campaign,2022-03-01,rosecam@cisco.com,2021-12-06,NaN,2022-03-03 18:03 PM Eastern,False,...,1,1,M1 (Coverage Program),Operations,ANY,Actual,2022,Q2,202204,2022-Q2


In [358]:
df.to_pickle(r'D:\Compass Trackers\TrackerData.pkl')
df.to_excel(r'D:\Compass Trackers\Tracker Data_' + today_file + '.xlsx', index=False)
# df.to_csv(hyper_csv_file_path_tracker, index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Compass Trackers\\TrackerData.pkl'

In [ ]:
df.loc[(df['Method'].str.contains('M1', na=False, case=False)) & (df['ALERT_BELOW_25'] == 1)][['Create Date','Customer Name','Compass Advisor','Readout Delivery Status','LPM_PLUS_ENGAGED','ADVISOR_NOTES','LPM_NOTES']]

In [ ]:
Done

In [ ]:
len(df.loc[df['Request ID'].isnull()]['Request ID'].unique())

In [ ]:
# df.loc[df['Request ID'].isin(['202120216643','202120216987'])][['Request ID','Requester Name','Create Date','GUID','CR Party ID','SAV ID','Customer Name','Lvl1','Lvl2 (Region)','Compass Campaign Name']]

In [ ]:
today = dt.datetime.today()

In [ ]:
today

### Create the COMPASS GENERATED file for Tableau

In [ ]:
compass_generated = df.loc[(df['Request FY'].isin(['2021','2022'])) & (df['DELIVERABILITY']=='Good') & (df['REMOVE_FROM_COUNT'] == True)][['Request ID','Create Date','Customer Name','GUID','CR Party ID', 'CR Party Name', 'CAV BU ID', 'CAV ID', 
                                                                                                                                'Deal ID', 'SAV ID', 'Contract ID', 'Inventory Name', 'Appliance ID', 'Requester Name','Requester Function','Campaign','Partner Name', 'Partner Deal','Status', 
                                                                                                                                'Lvl1', 'Lvl2 (Region)', 'Lvl3','Request Type', 'Date Completed', 'Compass Advisor','Readout_Date','Readout Delivery Status', 'Items Recommended','HashedReadout','CONCIERGE_ADVISOR_ASSIGNED',
                                                                                                                                'CONCIERGE_READOUT_DATE', 'CONCIERGE_READOUT_RESULT', 'Report Group', 'Request FY', 'Request FH','Request FQ', 'Request FM','Request FWOFQ','Request FWOFY','Readout FY','Readout FH','Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 
                                                                                                                                'Recipient Confirmation','RR (internal)','is_Targeting_Campaign', 'Campaign Type', 'Targeting Campaign Type', 'REMOVE_FROM_COUNT',]]#.pivot_table(values='Customer Name', columns='Deliverability', aggfunc='count')

In [ ]:
compass_generated.columns

In [ ]:
compass_generated.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\CompassGenerated.xlsx', sheet_name='CompassGenerated',  index=False)

### Readouts to Publish to Tableau

In [ ]:
readouts_for_tableau_path = r'C:\Users\phsheari\Documents\Compass Tableau Data\Readouts.xlsx'
undelivered_readouts_for_tableau_path = r'C:\Users\phsheari\Documents\Compass Tableau Data\UndeliveredReadouts.xlsx'

In [ ]:
_11months_ago = today + dt.timedelta(days = -330)

In [ ]:
# df['Date Readout Done'] = pd.to_datetime(df['Date Readout Done'])

In [ ]:
# readouts = df.loc[(df['Date Readout Done'].notnull()) & (df['Readout Delivery Status'].notnull())][['Request ID', 'Sheet Name', 'Previous Request ID', 'Create Date', 'Customer Name', 'GUID', 'CR Party ID','CAV ID', 'Deal ID', 'Requester Name',
#                                                                                                    'Partner Name', 'Compass Campaign Name','Status', 'Lvl1', 'Lvl2 (Region)', 'Lvl3','COMPASS URL 1.0', 'COMPASS PPT URL','Compass Concierge Requested',
#                                                                                                    'RR (internal)', 'Compass Advisor', 'BDM Notes', 'Items Recommended', 'Date Readout Done', 'Readout Delivery Status', 'Reason Declined', 'EXT_BDM_ASSIGNED',
#                                                                                                    'EXT_DATE_READOUT_DONE', 'EXT_READOUT_RESULT', 'Forecast Stage','Projected Booking ($,000)','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year',
#                                                                                                    'Deliverability','Recipient Confirmation', 'Request FY', 'Request FQ', 'Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY', 'Readout FY', 
#                                                                                                    'Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 'Campaign Type','IR_BCS', 'IR_EC', 'IR_DNAC', 'IR_SSPT', 'IR_ST', 'IR_SNTC', 
#                                                                                                    'IR_IRR', 'IR_PR', 'IR_BRW', 'TC_Priority_Recommend','Squeezed','Prepend']]

In [ ]:
# readouts['Prehash_Entity'] = readouts.apply(lambda x: x['Prepend'] + x['FakeID'], axis=1)

In [ ]:
# readouts[['Prehash_Entity','GUID','CAV ID','Customer Name']]

In [ ]:
df.loc[:,df.columns.str.contains('COMPASS URL', na=False, case=False)]

In [ ]:
readouts_publish_to_tableau  = df.loc[(df['Date Readout Done'].notnull()) & 
                                 ~df['Readout Delivery Status'].isin(['Declined','Postponed']) & 
                                 (df['GUID'].notnull() | df['CAV ID'].notnull()) &
                                 (df['Date Readout Done'] <= today) & 
                                 (df['REMOVE_FROM_COUNT'] == True) &
                                 (df['DELIVERABILITY']=='Good')][['Request ID', 'Create Date', 'Customer Name', 'GUID', 'CR Party ID','CAV ID', 'SAV ID','Deal ID', 'Requester Name',
                                                                  'Partner Name','Partner Deal', 'Campaign', 'Lvl1', 'Lvl2 (Region)', 'Lvl3','COMPASS URL', 'COMPASS PPT URL','RR (internal)', 'Report Group', 'Compass Advisor', 'Items Recommended', 'Date Readout Done', 'Readout Delivery Status', 
                                                                  'CONCIERGE_ADVISOR_ASSIGNED', 'CONCIERGE_READOUT_DATE', 'CONCIERGE_READOUT_RESULT', 'Forecast Stage','Opportunity Value','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year', 'TC_Priority_Recommend', 
                                                                  'Recipient Confirmation', 'Request FY', 'Request FH','Request FQ', 'Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY', 'Readout FY','Readout FH','Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 
                                                                  'is_Targeting_Campaign', 'Campaign Type','Targeting Campaign Type','Prehash_Entity','Entity_to_hash','HashedReadout','FakeID','REMOVE_FROM_COUNT']]

In [ ]:
def item_recommended_count(item_name, items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif item_name in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def bcs_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'BCS' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def ec_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'EC' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def irr_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'IRR' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def dnac_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'DNA' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def sspt_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'SSPT' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def sntc_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'SNTC' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def st_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'ST' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def pr_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'PR' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def brw_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 0
    elif 'BRW' in items_recommended:
        return 1
    else:
        return 0

In [ ]:
def NULL_recommended_count(items_recommended):
    if pd.isnull(items_recommended):
        return 1
    else:
        return 0

In [ ]:
# readouts_publish_to_tableau['HashEntity'] = readouts_publish_to_tableau.apply(lambda x: make_hash_entity(x['Prehash_Entity'], x['Compass Advisor'],x['Date Readout Done'], x['Items Recommended']), axis=1) #x['COMPASS URL 1.0'], 

In [ ]:
readouts_publish_to_tableau['Targeting Campaign Type'].unique()

In [ ]:
# readouts_publish_to_tableau['HashedReadout'] = readouts_publish_to_tableau.apply(lambda x: hash_the_readout(x['HashEntity']), axis=1 )

In [ ]:
# readouts_publish_to_tableau.HashedReadout.nunique()

In [ ]:
readouts_publish_to_tableau['IRC_BCS']  = readouts_publish_to_tableau['Items Recommended'].apply(bcs_recommended_count)
readouts_publish_to_tableau['IRC_EC']   = readouts_publish_to_tableau['Items Recommended'].apply(ec_recommended_count)
readouts_publish_to_tableau['IRC_IRR']  = readouts_publish_to_tableau['Items Recommended'].apply(irr_recommended_count)
readouts_publish_to_tableau['IRC_DNAC'] = readouts_publish_to_tableau['Items Recommended'].apply(dnac_recommended_count)
readouts_publish_to_tableau['IRC_SSPT'] = readouts_publish_to_tableau['Items Recommended'].apply(sspt_recommended_count)
readouts_publish_to_tableau['IRC_ST']   = readouts_publish_to_tableau['Items Recommended'].apply(st_recommended_count)
readouts_publish_to_tableau['IRC_SNTC'] = readouts_publish_to_tableau['Items Recommended'].apply(sntc_recommended_count)
readouts_publish_to_tableau['IRC_PR']   = readouts_publish_to_tableau['Items Recommended'].apply(pr_recommended_count)
readouts_publish_to_tableau['IRC_BRW']  = readouts_publish_to_tableau['Items Recommended'].apply(brw_recommended_count)
readouts_publish_to_tableau['IRC_NULL'] = readouts_publish_to_tableau['Items Recommended'].apply(NULL_recommended_count)

In [ ]:
readouts_publish_to_tableau.to_excel(readouts_for_tableau_path,sheet_name='TableauReadoutData', index=False)
readouts_publish_to_tableau.to_excel(r'C:\Users\phsheari\Documents\Compass Readouts\Compass Readouts.xlsx',sheet_name='TableauReadoutData', index=False)

In [ ]:
readouts_publish_to_tableau.groupby('REMOVE_FROM_COUNT')['Request ID'].count()

In [ ]:
latam_readouts1 = readouts_publish_to_tableau['Lvl2 (Region)']=='LATIN AMERICA'
latam_readouts3 = readouts_publish_to_tableau['Date Readout Done'] <= '2021-05-07'
latam_readouts2 = readouts_publish_to_tableau['Date Readout Done'] > '2020-10-07'

In [ ]:
latam_readouts = readouts_publish_to_tableau.loc[(latam_readouts1 & latam_readouts2 & latam_readouts3)][['Request ID','Date Readout Done','Customer Name','GUID','Compass Advisor','Readout Delivery Status','IRC_BCS','IRC_SSPT','IRC_EC','IRC_ST','IRC_SNTC']]

In [ ]:
#latam_readouts.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\1_AMERICAS\FY21Q4 Data\Latam Readouts.xlsx',index=False)

In [ ]:
readouts_publish_to_tableau.loc[readouts_publish_to_tableau['Customer Name'].str.contains('kfh', na=False, case=False)]

In [ ]:
readouts_not_done_condition1 = ((df['Readout Delivery Status'].isnull()) | (df['Readout Delivery Status'].isin(['Declined','Postponed']))) & (df['Items Recommended'].isnull())

In [ ]:
readouts_not_done_condition2 = df['DELIVERABILITY']=='Good'

In [ ]:
readouts_not_done_condition3 = ~(df['Request FY']==2021 & df['Campaign'].str.contains('growth initiative', case=False))

In [ ]:
readouts_not_done_condition4 = df['Request FY'].isin(['2021','2022'])

In [ ]:
readouts_not_done_condition5 = df['REMOVE_FROM_COUNT']==True

In [ ]:
readouts_not_done = df.loc[readouts_not_done_condition1 & readouts_not_done_condition2 & readouts_not_done_condition3 & readouts_not_done_condition4 & readouts_not_done_condition5][['Request ID', 'Create Date', 'Customer Name', 'GUID', 'CR Party ID','CAV ID', 'Deal ID','Requester Name',
                                                                                                   'Partner Name','Partner Deal', 'Campaign','Status', 'Lvl1', 'Lvl2 (Region)', 'Lvl3','COMPASS URL', 'COMPASS PPT URL','Compass Concierge Requested',
                                                                                                   'RR (internal)','Report Group','Compass Advisor', 'BDM Notes', 'Items Recommended', 'Date Readout Done', 'Readout Delivery Status', 'Reason Declined', 'CONCIERGE_ADVISOR_ASSIGNED',
                                                                                                   'CONCIERGE_READOUT_DATE', 'CONCIERGE_READOUT_RESULT',  'Forecast Stage','Opportunity Value','Target Fiscal Month', 'Target Fiscal Quarter', 'Target Fiscal Year',
                                                                                                   'DELIVERABILITY','Recipient Confirmation', 'Request FY', 'Request FH', 'Request FQ', 'Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY', 'Readout FY','Readout FH', 
                                                                                                   'Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY', 'is_Targeting_Campaign', 'Campaign Type', 'Targeting Campaign Type','IR_BCS', 'IR_EC', 'IR_DNAC', 'IR_SSPT', 'IR_ST', 'IR_SNTC', 
                                                                                                   'IR_IRR', 'IR_PR', 'IR_BRW', 'TC_Priority_Recommend','Entity_to_hash','HashedReadout','REMOVE_FROM_COUNT']]

In [ ]:
readouts_not_done

In [ ]:
### Readouts Scheduled
readouts_not_done.loc[(readouts_not_done['Date Readout Done']>= today) & (readouts_not_done['Request FY']>=str(today.year - 1))]

In [ ]:
### Readouts Not Done/ Not Scheduled / Not Declined
readouts_not_done.loc[(readouts_not_done['Date Readout Done'].isnull()) & (readouts_not_done['Request FY']>=str(today.year-1)) & ~(readouts_not_done['Readout Delivery Status'].isin(['Declined','Postponed']))].groupby(['Request FY','Campaign','Request FQ', 'Request FM',]).count()[['Request ID']]

In [ ]:
### Readouts Declined or Postponed
readouts_not_done.loc[readouts_not_done['Readout Delivery Status'].isin(['Declined','Postponed'])]

In [ ]:
readouts_not_done.to_excel(undelivered_readouts_for_tableau_path,sheet_name='Undelivered', index=False)
readouts_not_done.to_excel(r'C:\Users\phsheari\Documents\Compass Readouts\Undelivered Readouts.xlsx',sheet_name='Undelivered', index=False)

In [ ]:
readouts_not_done.columns

## Compass Partner Data

#### ---Partner Compass Delivered

In [ ]:
CURRENT_FISCAL_YEAR = today.year

In [ ]:
CURRENT_FISCAL_YEAR   #& (df['Request FY']==CURRENT_FISCAL_YEAR)   ##  ((df['Deal ID']==-44444) | df['Sheet Name'].str.contains('partner', case=False)) & 

In [ ]:
partner_model = (df['Deal ID']==-44444) & (df['Request FY'] == str(CURRENT_FISCAL_YEAR))  #[['Request ID','Request FY']]

In [ ]:
partner_compass_delivered = df.loc[partner_model][['Request ID','Create Date','Customer Name','GUID','Partner Name', 'Partner Deal', 'Lvl1','Lvl2 (Region)','Lvl3','Report Group','Compass Advisor','Date Readout Done', 'Request FY',  'Request FH', 
                                                   'Request FQ','Request FM', 'Request FWOFM', 'Request FWOFQ', 'Request FWOFY','Readout FY', 'Readout FH', 'Readout FQ','Readout FM', 'Readout FWOFM', 'Readout FWOFQ', 'Readout FWOFY']] 

In [ ]:
# partner_compass_delivered['GUID'] = partner_compass_delivered['GUID'].astype(int)

In [ ]:
partner_compass_delivered.sort_values(by=(['Lvl1','Partner Name','Lvl2 (Region)','Create Date','Request ID','Customer Name']), inplace=True)

In [ ]:
# partner_compass_delivered.groupby(['Request FY','Lvl1','Partner Name']).count()[['Request ID','Compass Advisor','Date Readout Done']]

#### ---Partner Readouts Delivered

In [ ]:
partner_readouts_delivered = df.loc[(df['Deal ID']==-44444) & 
#                                     (df['Request FY']==CURRENT_FISCAL_YEAR) & 
                                    (df['Readout Delivery Status'].str.contains('complete', case=False))  & 
                                    (df['DELIVERABILITY']=='Good')][['Request ID','Create Date','Customer Name','GUID','Partner Name','Partner Deal','Lvl1','Lvl2 (Region)','Compass Advisor','Date Readout Done','Readout Delivery Status','Items Recommended','Readout FY','Readout FH','Readout FQ','Readout FM',]]

In [ ]:
partner_readouts_delivered['GUID'] = partner_readouts_delivered['GUID'].astype(int)

In [ ]:
partner_readouts_delivered.sort_values(by=(['Lvl1','Partner Name','Create Date','Request ID','Customer Name']), inplace=True)

In [ ]:
partner_readouts_delivered.groupby(['Readout FY','Readout FQ','Readout FM']).count()['Request ID']

In [ ]:
partner_compass_delivered.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\PartnerData.xlsx',sheet_name='Partners', index=False)

In [ ]:
Done

## Compass Pipeline Data

In [ ]:
### Compass Pipeline Conditions
forecast_stages = ['1 - Prospect','2 - Qualification','3 - Proposal','4 - Agreement']
pipeline1 = (df['Forecast Stage'].isin(forecast_stages))
pipeline2 = (df['Deliverability']=='Good')
pipeline3 = ((df['Target Fiscal Year'] == str(CURRENT_FISCAL_YEAR)) | (df['Target Fiscal Year'] == str(CURRENT_FISCAL_YEAR + 1)))
pipeline4 = (df['Request FY'].isin([str(CURRENT_FISCAL_YEAR - 1),str(CURRENT_FISCAL_YEAR)]))

In [ ]:
compass_pipeline_data = df.loc[pipeline1 & pipeline2 & pipeline3 & pipeline4][['Request ID','Create Date','Customer Name','Requester Name','GUID','CAV ID','Deal ID', 'Lvl1','Lvl2 (Region)','Forecast Stage','Forecast Status','Opportunity Value','Target Fiscal Month','Target Fiscal Quarter','Target Fiscal Year']] 

In [ ]:
compass_pipeline_data['Deal ID'] = compass_pipeline_data['Deal ID'].astype(int)

In [ ]:
# compass_pipeline_data['GUID'] = compass_pipeline_data['GUID'].astype(int)

In [ ]:
compass_pipeline_data.loc[compass_pipeline_data['GUID'].isnull()]

In [ ]:
pd.pivot_table(data=compass_pipeline_data, values='Projected Booking ($,000)', index=['Target Fiscal Year','Target Fiscal Quarter'], columns='Forecast Stage', aggfunc=['sum','count'], fill_value = 0.0)

In [ ]:
compass_pipeline_data.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\CompassPipeline.xlsx',sheet_name='Pipeline', index=False)

In [ ]:
non_reporting_campaigns = ['accelerator','test','test request','proactive - ignore']

In [ ]:
## Compass Inventory Generated
compass_generated_inventory = df.loc[(df['Deliverability']=='Good') & ~(df['Campaign'].str.lower().isin(non_reporting_campaigns))][['Request ID', 'Create Date','Date Completed','Complete?','Date Readout Done','Customer Name','Requester Name','Requester Function','Partner Name','Partner Deal','Recipient Confirmation','Compass Campaign Name', 'Request FY', 'Lvl1','Lvl2 (Region)','Lvl3','Campaign Type','Record Type','REMOVE_FROM_COUNT','RR (internal)','Targeting Campaign Type']]

In [ ]:
compass_generated_inventory.loc[(compass_generated_inventory['Request FY']=='2021') & (compass_generated_inventory['REMOVE_FROM_COUNT']==True) & ~(compass_generated_inventory['RR (internal)']=='NO')]  #(compass_generated_inventory['Recipient Confirmation']=='Confirmed') & 

In [ ]:
compass_generated_inventory.to_excel(r'C:\Users\phsheari\Documents\Compass Tableau Data\Compass_Generated_Inventory.xlsx', sheet_name='Compass Generated', index=False)

In [ ]:
# if finish.weekday() == 4:
#     send_file = r'C:\Users\phsheari\Documents\Compass Pipeline' + "\\" + 'Compass_Maven_Bookings_Pipeline_Partners_ETL_File_' + today_file[:8] + '.xlsx'
#     with pd.ExcelWriter(send_file) as writer:  #endfilename
#         compass_pipeline_data.to_excel(writer, sheet_name = "Compass Pipeline Data", index=False)
#         compas_finbi_bookings.to_excel(writer, sheet_name = 'Compass FINBI Bookings', index=False)
#         partner_compass_delivered.to_excel(writer, sheet_name = "Partner Compass Delivered", index=False)
#         partner_readouts_delivered.to_excel(writer, sheet_name = "Partner Readouts Delivered", index=False)
#         finbi_partner_deals.to_excel(writer, sheet_name = 'Partner Bookings (from Compass)', index = False)   

### LATAM records from Smartsheet

In [ ]:
df.loc[(df['Lvl2 (Region)'].notnull()) & 
       (df['Lvl2 (Region)'].str.contains('LATIN AMERICA', case=False)) & 
       (df['Request FY'] == '2021')  & 
       ~(df['Campaign'].str.contains('growth', na=False, case=False))
      ].groupby(['Campaign Type','Targeting Campaign Type']).count()[['Request ID','Compass Advisor','Date Readout Done','Readout Delivery Status','Items Recommended','HashedReadout']]

In [ ]:
finish = dt.datetime.now()
elapsed = (finish - start)
print(elapsed/60.0)

In [ ]:
Done

## Healthcare records exercise

In [ ]:
h1 = df['Customer Name'].str.contains('medical', case=False)

In [ ]:
h2 = df['Customer Name'].str.contains('hospital', case=False)

In [ ]:
h3 = df['Customer Name'].str.contains('health', case=False)

In [ ]:
h4 = df['Customer Name'].str.contains('physician', case=False)

In [ ]:
h5 = df['Customer Name'].str.contains('pharma', case=False)

In [ ]:
h6 = df['Customer Name'].str.contains('laboratori', case=False)

In [ ]:
h7 = df['Customer Name'].str.contains('labs', case=False)

In [ ]:
h8 = df['Customer Name'].str.contains('pfizer', case=False)

In [ ]:
h9 = df['Customer Name'].str.contains('bayer', case=False)

In [ ]:
h10 = df['Customer Name'].str.contains('merck', case=False)

In [ ]:
h11 = df['Customer Name'].str.contains('roche', case=False)

In [ ]:
h12 = df['Customer Name'].str.contains('novartis', case=False)

In [ ]:
h13 = df['Customer Name'].str.contains('glaxo', case=False)

In [ ]:
h14 = df['Customer Name'].str.contains('johnson & johnson', case=False)

In [ ]:
h15 = df['Customer Name'].str.contains('sanofi', case=False)

In [ ]:
h16 = df['Customer Name'].str.contains('abbvie', case=False)

In [ ]:
h17 = df['Customer Name'].str.contains('zeneca', case=False)

In [ ]:
h18 = df['Customer Name'].str.contains('baxter', case=False)

In [ ]:
h19 = df['Customer Name'].str.contains('bausch', case=False)

In [ ]:
h20 = df['Customer Name'].str.contains('biogen', case=False)

In [ ]:
h21 = df['Customer Name'].str.contains('celgene', case=False)

In [ ]:
h22 = df['Customer Name'].str.contains('moderna', case=False)

In [ ]:
h23 = df['Customer Name'].str.contains('stryker', case=False)

In [ ]:
h24 = df['Customer Name'].str.contains('teva', case=False)

In [ ]:
h25 = df['Customer Name'].str.contains('medtronic', case=False)

In [ ]:
h26 = df['Customer Name'].str.contains('diagnostic', case=False)

In [ ]:
today = dt.datetime.now()

In [ ]:
last_90_days = today.date() + dt.timedelta(days=-90)

In [ ]:
last_90_days

In [ ]:
h_last_90 = (df['Create Date'] >= last_90_days)

In [ ]:
health2 = df.loc[(h1 | h2 | h3 | h4 | h5 | h6 | h8 | h9 | h10 | h11 | h12 | h13 | h14 | h15 | h16 | h17 | h18 | h19 | h20 | h21 | h22 | h23 | h24 | h25 | h26) & h_last_90][['Request ID','Create Date','Customer Name','GUID','Lvl1','Lvl2 (Region)','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
health2.shape

In [ ]:
health2.sort_values(by=(['Create Date','Customer Name']), inplace=True)

In [ ]:
health2.to_excel(r'C:\Users\phsheari\Documents\Francisco\Healthcare_Compass_Accounts3.xlsx', index=False)

In [ ]:
emma_scudder = df.loc[h_last_90 & (df['Deliverability']=='Good')][['Request ID','Create Date','Customer Name','GUID', 'CAV ID','CAV BU ID','SAV ID','CR Party ID','CR Party Name','Appliance ID','Contract ID', 'Lvl1','Lvl2 (Region)','Lvl3']]

In [ ]:
emma_scudder.to_excel(r'C:\Users\phsheari\Documents\Francisco\Emma_Scudder_Request_Last_90_Days_Compass.xlsx', index=False)

### Main Work Complete

In [ ]:
df[['Request ID','Sheet Name']].nunique()

In [ ]:
finish

In [ ]:
df['Deal ID'] = df['Deal ID'].apply(convert_nan_to_int_to_str)

In [ ]:
campaign_counts = pd.DataFrame(df.loc[df['Campaign].notnull()][['Lvl1','Lvl2 (Region)','Lvl3','Campaign','Requester Name','Create Date','Request FY','Request FQ']].value_counts())

In [ ]:
campaign_counts.rename(columns={0:'Counts'}, inplace=True)

In [ ]:
campaign_counts.sort_values(['Campaign', 'Request FY','Request FQ'], inplace=True)

In [ ]:
campaign_counts.reset_index(inplace=True)

In [ ]:
campaign_counts.columns

In [ ]:
campaign_counts.to_csv(r'C:\Users\phsheari\Desktop\campaigns.csv', index=False)

In [ ]:
# bigdict = pre_dict.to_dict('records')

In [ ]:
# bigdict

In [ ]:
# dic = {}
# for x in bigdict:
#     dic[x['Request ID']] = x['Requester Name']

In [ ]:
# dic

In [ ]:
# len(dic.items()) #['Compass Campaign Name']

In [ ]:
today = dt.datetime.today().date()

In [ ]:
today

In [ ]:
_30days_ago = today + dt.timedelta(days=-30)

In [ ]:
_90days_ago = today + dt.timedelta(days=-90)

In [ ]:
_90days_ago

In [ ]:
cond0 = df['Compass Campaign Name'].notnull()
cond11 = df['Compass Campaign Name'].str.contains('target', case=False)
cond12 = df['Compass Campaign Name'].str.contains('success', case=False)
cond13 = df['Compass Campaign Name'].str.contains('bcs', case=False)
cond3 = df['Date Readout Done'] >= _30days_ago
cond4 = (df['Date Readout Done'] > _90days_ago) & (df['Date Readout Done'] <= _30days_ago)

In [ ]:
cond5 =  df['Create Date'] >= _30days_ago
cond6 = df['Date Readout Done'].isnull()
cond7 = (df['RR (internal)'].isnull() | df['RR (internal)'].str.lower()=='yes')
cond8 = (df['RR (internal)'].isnull() | df['RR (internal)'].str.lower()=='no')

In [ ]:
_30_day_data = df.loc[cond3 & cond7][['Request ID', 'Create Date', 'Sheet Name','Requester Name','Customer Name','GUID','Compass Campaign Name','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
new_last_30_days = df.loc[cond0 & (cond11 | cond12 | cond13) & cond5 & cond6][['Request ID', 'Create Date', 'Sheet Name','Requester Name','Customer Name','GUID','Compass Campaign Name','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
new_entries_to_cancel = pd.merge(new_last_30_days, _30_day_data, how ='inner', left_on = ['GUID','Customer Name'], right_on = ['GUID','Customer Name'])[['Request ID_x', 'Create Date_x', 'Sheet Name_x','Requester Name_x','Customer Name','GUID','Compass Campaign Name_x','Compass Advisor_x','Date Readout Done_x', 'Readout Delivery Status_x']]

In [ ]:
newcols = []
for col in new_entries_to_cancel.columns:
    newcols.append(col.replace('_x',''))
new_entries_to_cancel.columns = newcols

In [ ]:
new_entries_to_cancel.drop_duplicates(inplace=True)

In [ ]:
new_entries_to_cancel.sort_values(by=['Sheet Name','Request ID'], inplace=True, ignore_index=True)

In [ ]:
new_entries_to_cancel['Requester Name'] = new_entries_to_cancel['Requester Name'].apply(lambda x: None if x == 'nan@cisco.com' else x)

In [ ]:
new_entries_to_cancel

In [ ]:
_90day_data = df.loc[cond0 & (cond11 | cond12 | cond13) & cond4 & cond8][['Request ID', 'Create Date', 'Sheet Name','Requester Name','Customer Name','GUID','Compass Campaign Name','Compass Advisor','Date Readout Done', 'Readout Delivery Status']]

In [ ]:
new_entries_to_schedule = pd.merge(new_last_30_days, _90day_data, how ='inner', left_on = ['GUID','Customer Name'], right_on = ['GUID','Customer Name'])[['Request ID_x', 'Create Date_x','Sheet Name_x','Requester Name_x','Customer Name','GUID','Compass Campaign Name_x','Compass Advisor_x','Date Readout Done_x', 'Readout Delivery Status_x']]

In [ ]:
newcols_sched = []
for col in new_entries_to_schedule.columns:
    newcols_sched.append(col.replace('_x',''))
new_entries_to_schedule.columns = newcols_sched

In [ ]:
new_entries_to_schedule.drop_duplicates(inplace=True)

In [ ]:
new_entries_to_schedule.sort_values(by=['Sheet Name','Request ID'], inplace=True, ignore_index=True)

In [ ]:
# new_entries_to_schedule['Requester Name'] = new_entries_to_schedule['Requester Name'].apply(lambda x: None if x.startswith('nan') else x)

In [ ]:
new_entries_to_schedule

In [ ]:
with pd.ExcelWriter(r'C:\\Users\\phsheari\\Documents\\Compass Scheduling' + "\\" + 'Stacey_To_Schedule_' + today_file[:8] + '.xlsx') as writer:  #endfilename
    new_entries_to_cancel.to_excel(writer, sheet_name = 'Entries for NO', index = False)
    new_entries_to_schedule.to_excel(writer, sheet_name = 'Entries to Schedule', index = False)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

In [ ]:
partner_list = list(df.loc[df['Partner Name'].notnull()]['Partner Name'].str.lower()\
                                                    .replace({'. ':'',
                                                              '.':''})\
                                                    .unique()
                   )

In [ ]:
finbi_df.columns

In [ ]:
finbi_df.loc[finbi_df['Partner Deal']==True] [['Sales Level 1','Sales Level 2','GUID Name','CAV Name','Partner','Fiscal Quarter ID','ProductType','CX Product Category','TCV']]

In [ ]:
finbi_partners_booked = pd.pivot_table(finbi_df.loc[finbi_df['Partner Deal']==True][['Sales Level 1','Sales Level 2','GUID Name','CAV Name','Campaign','Partner','Fiscal Quarter ID','ProductType','CX Product Category','TCV']], index=['Sales Level 1','Sales Level 2','Partner','GUID Name','CAV Name','Campaign','ProductType','CX Product Category',], values='TCV', fill_value='999', columns=['Fiscal Quarter ID'], margins_name='Total')

In [ ]:
finbi_partners_booked.reset_index(inplace=True)

In [ ]:
finbi_partners_booked